In [1]:
import pandas as pd
import os 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import re
import pandas as pd
import string

In [2]:
#reading all three files
agency = pd.read_csv("/project/merge/AGENCY.csv")
client = pd.read_csv("/project/notebooks/client_alg.csv")
rfm = pd.read_csv("/project/notebooks/rfm_alg.csv")

# client

In [3]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 23 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Brief                                   162 non-null    object 
 1   Category                                162 non-null    object 
 2   Client Name                             162 non-null    object 
 3   Status                                  162 non-null    object 
 4   Lead                                    162 non-null    object 
 5   Shortlisted Agencies                    162 non-null    object 
 6   Winning Agency                          162 non-null    object 
 7   Region                                  162 non-null    object 
 8   Agency 1                                162 non-null    object 
 9   Agency 2                                162 non-null    object 
 10  Agency 3                                162 non-null    object

In [4]:
# Columns to drop
columns_to_drop = ['Agency 1', 'Agency 2', 'Agency 3', 'Agency 4', 'Agency 5', 'Agency 6', 'Lead', 
                   'shortlisted_agency_1_interaction_count', 'shortlisted_agency_2_interaction_count',
                   'shortlisted_agency_3_interaction_count', 'shortlisted_agency_4_interaction_count',
                   'shortlisted_agency_5_interaction_count', 'shortlisted_agency_6_interaction_count', 
                  'Tags', 'Category']

# Drop specified columns
client = client.drop(columns=columns_to_drop)


In [5]:
client.head()

,Brief,Client Name,Status,Shortlisted Agencies,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,"Boost Awards,Reed Words",Boost Awards,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,"Viewpoint Studios,Shoreditch Design Studio",Shoreditch Design Studio,united kingdom,2,1.0
2,Prop Dev (EV and CC Insurance),AXA,Started,"Blue Feathers,Daughter Studio,CreateFuture",Daughter Studio,united kingdom,3,1.0
3,Digital Claims,AXA,Contracting,"Oxara London,Secret Source Marketing",Secret Source Marketing,united kingdom,2,1.0
4,Connected Car prop dev sprint,AXA,Brief Abandoned,"CreateFuture,HACKMASTERS,Loomery",1,united kingdom,3,1.0


In [6]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Brief                  162 non-null    object 
 1   Client Name            162 non-null    object 
 2   Status                 162 non-null    object 
 3   Shortlisted Agencies   162 non-null    object 
 4   Winning Agency         162 non-null    object 
 5   Region                 162 non-null    object 
 6   Agency Count           162 non-null    int64  
 7   Client_WA_Interaction  162 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 10.2+ KB


In [7]:
client['Winning Agency'] = client['Winning Agency'].replace('1', "unknown")
client["Winning Agency"].value_counts()

unknown                            42
Shoreditch Design Studio           12
Buddy Media                         8
GH05T Ltd.                          7
Path Ventures                       7
Reed Words                          6
NMD+                                6
Innovate Live                       5
Hedgehog Lab                        4
Viewpoint Studios                   3
Sketch & Co                         3
Raw.Studio                          3
Good Innovation                     3
PALO IT                             3
Apex1                               2
In Marketing We Trust               2
Nimbletank                          2
How To Impact                       2
Wonderful                           2
Studiospace                         2
Quill Peak Consulting               2
The Bang                            2
Orange Panther Collective           2
UNPITCHD                            2
Houston Group                       2
Daughter Studio                     2
MyMavins    

In [8]:
# import string
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# # Function to preprocess text
# def preprocess_text(text):
#     # If the text is null (i.e., no information), return it as is
#     if pd.isnull(text):
#         return text

#     # Convert text to lowercase
#     text = text.lower()

#     # Replace new line characters with space
#     text = text.replace('\n', ' ')

#     # Remove punctuation but preserve commas, numbers, and parentheses
#     punct_except_comma_numbers_parentheses = string.punctuation.replace(',', '') # preserve commas
#     punct_except_comma_numbers_parentheses = punct_except_comma_numbers_parentheses.replace('.', '') # preserve periods for numbers like 2.0, 3.5, etc.
#     punct_except_comma_numbers_parentheses = punct_except_comma_numbers_parentheses.replace('(', '') # preserve (
#     punct_except_comma_numbers_parentheses = punct_except_comma_numbers_parentheses.replace(')', '') # preserve )
#     text = text.translate(str.maketrans("", "", punct_except_comma_numbers_parentheses))

#     # Tokenize text into words
#     words = word_tokenize(text)

#     # Remove stop words
#     stop_words = set(stopwords.words('english'))
#     words = [word for word in words if word not in stop_words]

#     # Join words back into text
#     processed_text = ' '.join(words)

#     # Strip leading/trailing white spaces
#     processed_text = processed_text.strip()

#     return processed_text

# # Columns to preprocess in the client df
# text_columns = ['Brief', 'Client Name', 'Status', 'Shortlisted Agencies',
#                 'Winning Agency', 'Region']

# # Apply preprocessing to text columns
# for column in text_columns:
#     client[column] = client[column].apply(preprocess_text)

In [9]:
client.head()

,Brief,Client Name,Status,Shortlisted Agencies,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,"Boost Awards,Reed Words",Boost Awards,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,"Viewpoint Studios,Shoreditch Design Studio",Shoreditch Design Studio,united kingdom,2,1.0
2,Prop Dev (EV and CC Insurance),AXA,Started,"Blue Feathers,Daughter Studio,CreateFuture",Daughter Studio,united kingdom,3,1.0
3,Digital Claims,AXA,Contracting,"Oxara London,Secret Source Marketing",Secret Source Marketing,united kingdom,2,1.0
4,Connected Car prop dev sprint,AXA,Brief Abandoned,"CreateFuture,HACKMASTERS,Loomery",unknown,united kingdom,3,1.0


# Reasons for Matching data

In [10]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 20 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Client                                              280 non-null    object 
 1   Brief                                               280 non-null    object 
 2   Agency                                              280 non-null    object 
 3   Project extension 
with same agency                 280 non-null    int64  
 4   Client selected agency
themselves from marketplace  280 non-null    float64
 5   Right geography
 / time zone                        280 non-null    int64  
 6   Strong skill
set match                              280 non-null    int64  
 7   Deep / relevant
sector expertise                    280 non-null    int64  
 8   Good cultural fit                                   280 non-null    int64  
 9  

In [11]:
rfm.head()

,Client,Brief,Agency,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,Right geography\n / time zone,Strong skill\nset match,Deep / relevant\nsector expertise,Good cultural fit,Right price level,Strong track record,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Match Score
0,Aegon,Senior UI Designer,Hedgehog Lab,0,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0.558140
1,Aegon,UX 2 month Resourcing,Shoreditch Design Studio,0,0.0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0.790698
2,Allego,EV Location Partner Campaign,GH05T Ltd.,0,0.0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0.790698
3,Allego,EV Location Partner Campaign,Buddy Media,0,0.0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0.651163
4,Allego,EV Location Partner Campaign,bbp,0,0.0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0.325581


In [12]:
# # Columns to drop
# columns_to_drop = ["Lead_George Patten", "Lead_Gideon Hyde", "Lead_Hannah Fraser",
#                    "Lead_Kate Walker", "Lead_Paul Bowman", "Lead_Peter Sayburn", 
#                   "Lead_Phil Kohler", "Lead_Robin Scarborough"]

# # Drop specified columns
# rfm = rfm.drop(columns=columns_to_drop)

In [13]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 20 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Client                                              280 non-null    object 
 1   Brief                                               280 non-null    object 
 2   Agency                                              280 non-null    object 
 3   Project extension 
with same agency                 280 non-null    int64  
 4   Client selected agency
themselves from marketplace  280 non-null    float64
 5   Right geography
 / time zone                        280 non-null    int64  
 6   Strong skill
set match                              280 non-null    int64  
 7   Deep / relevant
sector expertise                    280 non-null    int64  
 8   Good cultural fit                                   280 non-null    int64  
 9  

In [14]:
rfm.head()

,Client,Brief,Agency,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,Right geography\n / time zone,Strong skill\nset match,Deep / relevant\nsector expertise,Good cultural fit,Right price level,Strong track record,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Match Score
0,Aegon,Senior UI Designer,Hedgehog Lab,0,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0.558140
1,Aegon,UX 2 month Resourcing,Shoreditch Design Studio,0,0.0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0.790698
2,Allego,EV Location Partner Campaign,GH05T Ltd.,0,0.0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0.790698
3,Allego,EV Location Partner Campaign,Buddy Media,0,0.0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0.651163
4,Allego,EV Location Partner Campaign,bbp,0,0.0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0.325581


# agency 

In [15]:
agency.head()

,Agency,Skills,Industries,Service Description,Vision,Industry Experience,Agency short description,Agency services,Any other services,Locations
0,5 Ways To Innovate,Growth & Innovation Strategy,"Financial Services,Technology, Information and...",We help corporates unlock the capability to re...,NaN,NaN,"We’re on a mission to help 1,000 corporates re...","Strategic Design, Leadership Development & Coa...","Innovation Strategy, Corporate Innovation",Australia
1,Arktic Fox,"Data Strategy, Digital Strategy, eCommerce & M...","Alcoholic Beverages, Education, Fashion, FMCG,...",Arktic Fox is a digital & eCommerce advisory a...,NaN,NaN,"We are on a mission to help marketing, digital...",CRM & Loyalty,"eCommerce strategy, Digital Transformation, Di...",Australia
2,Boost Awards,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,NaN,NaN,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,United Kingdom
3,Bread Agency,"Digital Strategy, Social Media, Creative Produ...","Alcoholic Beverages, Automotive, Beauty, Enter...",Social media agency making content people want...,NaN,NaN,Social media agency making content people want...,"Research & Insight, Creative & Production, Soc...","Influencer Marketing, Corporate Reputation, Co...",Australia
4,Chs,NaN,NaN,"Award-Winning, Digital First, Full Service Agency",NaN,NaN,"Award-Winning, Digital First, Full Service Agency","Brand Strategy, Creative & Production, Digital...",Photography and Film,"United Kingdom, United States"


In [16]:
agency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Agency                    213 non-null    object
 1   Skills                    197 non-null    object
 2   Industries                197 non-null    object
 3   Service Description       209 non-null    object
 4   Vision                    78 non-null     object
 5   Industry Experience       77 non-null     object
 6   Agency short description  212 non-null    object
 7   Agency services           212 non-null    object
 8   Any other services        141 non-null    object
 9   Locations                 212 non-null    object
dtypes: object(10)
memory usage: 16.8+ KB


## creating a merge as per sheet

In [17]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Brief                  162 non-null    object 
 1   Client Name            162 non-null    object 
 2   Status                 162 non-null    object 
 3   Shortlisted Agencies   162 non-null    object 
 4   Winning Agency         162 non-null    object 
 5   Region                 162 non-null    object 
 6   Agency Count           162 non-null    int64  
 7   Client_WA_Interaction  162 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 10.2+ KB


In [18]:
client.head()

,Brief,Client Name,Status,Shortlisted Agencies,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,"Boost Awards,Reed Words",Boost Awards,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,"Viewpoint Studios,Shoreditch Design Studio",Shoreditch Design Studio,united kingdom,2,1.0
2,Prop Dev (EV and CC Insurance),AXA,Started,"Blue Feathers,Daughter Studio,CreateFuture",Daughter Studio,united kingdom,3,1.0
3,Digital Claims,AXA,Contracting,"Oxara London,Secret Source Marketing",Secret Source Marketing,united kingdom,2,1.0
4,Connected Car prop dev sprint,AXA,Brief Abandoned,"CreateFuture,HACKMASTERS,Loomery",unknown,united kingdom,3,1.0


In [19]:
# Split the shortlisted agencies in the client dataset
client['Shortlisted Agencies'] = client['Shortlisted Agencies'].str.split(',')

# Explode the client dataset based on the shortlisted agencies
client = client.explode('Shortlisted Agencies')
client

,Brief,Client Name,Status,Shortlisted Agencies,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0
0,Awards applications x5\n,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0
2,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0
...,...,...,...,...,...,...,...,...
159,Social Media Veezu,Veezu,Started,GH05T Ltd.,GH05T Ltd.,united kingdom,3,2.0
159,Social Media Veezu,Veezu,Started,Nonsensical,GH05T Ltd.,united kingdom,3,2.0
159,Social Media Veezu,Veezu,Started,Spin Brands,GH05T Ltd.,united kingdom,3,2.0
160,SME engagement,Wave,Archived,NMD+,NMD+,united kingdom,1,2.0


In [20]:
client.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 161
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Brief                  296 non-null    object 
 1   Client Name            296 non-null    object 
 2   Status                 296 non-null    object 
 3   Shortlisted Agencies   296 non-null    object 
 4   Winning Agency         296 non-null    object 
 5   Region                 296 non-null    object 
 6   Agency Count           296 non-null    int64  
 7   Client_WA_Interaction  296 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.8+ KB


In [21]:
client.head()

,Brief,Client Name,Status,Shortlisted Agencies,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0
0,Awards applications x5\n,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0
1,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0
2,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0


In [22]:
client = client.reset_index(drop=True)
client

,Brief,Client Name,Status,Shortlisted Agencies,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0
1,Awards applications x5\n,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0
2,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0
3,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0
4,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0
...,...,...,...,...,...,...,...,...
291,Social Media Veezu,Veezu,Started,GH05T Ltd.,GH05T Ltd.,united kingdom,3,2.0
292,Social Media Veezu,Veezu,Started,Nonsensical,GH05T Ltd.,united kingdom,3,2.0
293,Social Media Veezu,Veezu,Started,Spin Brands,GH05T Ltd.,united kingdom,3,2.0
294,SME engagement,Wave,Archived,NMD+,NMD+,united kingdom,1,2.0


In [49]:
# client.to_csv("client_exploded.csv")

In [23]:
rfm.head()

,Client,Brief,Agency,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,Right geography\n / time zone,Strong skill\nset match,Deep / relevant\nsector expertise,Good cultural fit,Right price level,Strong track record,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Match Score
0,Aegon,Senior UI Designer,Hedgehog Lab,0,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0.558140
1,Aegon,UX 2 month Resourcing,Shoreditch Design Studio,0,0.0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0.790698
2,Allego,EV Location Partner Campaign,GH05T Ltd.,0,0.0,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0.790698
3,Allego,EV Location Partner Campaign,Buddy Media,0,0.0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0.651163
4,Allego,EV Location Partner Campaign,bbp,0,0.0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0.325581


In [24]:
# Rename the columns in the client dataframe to match the column names in the rfm dataframe
client = client.rename(columns={"Brief Name": "Brief",
                                "Shortlisted Agencies": "Agency",
                                "Client Name": "Client"})

In [25]:
client

,Brief,Client,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5\n,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0
1,Awards applications x5\n,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0
2,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0
3,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0
4,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0
...,...,...,...,...,...,...,...,...
291,Social Media Veezu,Veezu,Started,GH05T Ltd.,GH05T Ltd.,united kingdom,3,2.0
292,Social Media Veezu,Veezu,Started,Nonsensical,GH05T Ltd.,united kingdom,3,2.0
293,Social Media Veezu,Veezu,Started,Spin Brands,GH05T Ltd.,united kingdom,3,2.0
294,SME engagement,Wave,Archived,NMD+,NMD+,united kingdom,1,2.0


In [26]:
# # Strip leading/trailing white space from data in specified columns in both dataframes
# client[['Brief', 'Agency', 'Client']] = client[['Brief', 'Agency', 'Client']].apply(lambda x: x.str.strip())
# rfm[['Brief', 'Agency', 'Client']] = rfm[['Brief', 'Agency', 'Client']].apply(lambda x: x.str.strip())

# # Convert all text in specified columns to lower case in both dataframes
# client[['Brief', 'Agency', 'Client']] = client[['Brief', 'Agency', 'Client']].apply(lambda x: x.str.lower())
# rfm[['Brief', 'Agency', 'Client']] = rfm[['Brief', 'Agency', 'Client']].apply(lambda x: x.str.lower())

# # Fill NA/NaN values with some consistent value in both dataframes
# # Here I'm filling with empty string, but you may want to choose another value
# client[['Brief', 'Agency', 'Client']] = client[['Brief', 'Agency', 'Client']].fillna('')
# rfm[['Brief', 'Agency', 'Client']] = rfm[['Brief', 'Agency', 'Client']].fillna('')

# # Remove special characters from the specified columns in both dataframes
# # This is assuming special characters are not supposed to be part of the data in these columns
# client[['Brief', 'Agency', 'Client']] = client[['Brief', 'Agency', 'Client']].apply(lambda x: x.str.replace('[^\w\s]', ''))
# rfm[['Brief', 'Agency', 'Client']] = rfm[['Brief', 'Agency', 'Client']].apply(lambda x: x.str.replace('[^\w\s]', ''))



In [27]:
# Removing newline characters from the "Brief" column in the client dataset
client["Brief"] = client["Brief"].str.strip()
client

,Brief,Client,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction
0,Awards applications x5,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0
1,Awards applications x5,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0
2,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0
3,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0
4,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0
...,...,...,...,...,...,...,...,...
291,Social Media Veezu,Veezu,Started,GH05T Ltd.,GH05T Ltd.,united kingdom,3,2.0
292,Social Media Veezu,Veezu,Started,Nonsensical,GH05T Ltd.,united kingdom,3,2.0
293,Social Media Veezu,Veezu,Started,Spin Brands,GH05T Ltd.,united kingdom,3,2.0
294,SME engagement,Wave,Archived,NMD+,NMD+,united kingdom,1,2.0


In [28]:
# Removing leading and trailing whitespace from the "Brief" and "Agency" columns in the client dataset
client['Brief'] = client['Brief'].str.strip()
client['Agency'] = client['Agency'].str.strip()

# Removing leading and trailing whitespace from the "Brief" and "Agency" columns in the rfm dataset
rfm['Brief'] = rfm['Brief'].str.strip()
rfm['Agency'] = rfm['Agency'].str.strip()

In [29]:
# Brief Column
# Checking for differences in the "Brief" column
different_briefs = set(client['Brief']).symmetric_difference(set(rfm['Brief']))
if different_briefs:
    print("\nDifferences in 'Brief' column:")
    print(different_briefs)


Differences in 'Brief' column:
{'Gen AI smoke Tests', 'Sustainable Home prototype', '3 Yr OneApp UX/UI Deal', 'Employer Value Proposition Creative Partner', 'Electrification Qual Research', '2+ Yr OneApp UX/UI Deal', 'Panthera OneApp Ongoing UX/UI Support', 'CBUS Claims Sprint', 'Shepper B2B Marketing', 'IMWT Scope Extension', 'James Hardie PDF to Microsite Build', 'FIFA website', 'Escape Travel eDM Build', 'XD team support and capability build', 'MA Financial Website - US Version', 'Project Trident', 'Dementia Proposition', 'Retirement Modelling Tool', 'Research help', 'Brand Escape Room Production', 'SEO Aviva', 'Book design and production', 'Digital Marketing Campaign', 'Soho Blitz Day Media Support', 'MOCO 2023 - Media Budget', 'Call centre automation', 'MOCO 2023 - Agency fee', 'Sustainability Copywriting  - Extension', 'Rebrand & Research', 'Equity Release campaign', 'Social Media support', 'Escape Room York', 'Sales Playbook', 'Expert Videos x5 Bristol', 'UX Design System', 'Co

In [30]:
# Finding values in the 'Brief' column that are present in one dataset but not the other
missing_briefs_in_client = set(client['Brief']) - set(rfm['Brief'])
missing_briefs_in_rfm = set(rfm['Brief']) - set(client['Brief'])
print("\nMissing 'Brief' values in the client dataset:", missing_briefs_in_client)
print("Missing 'Brief' values in the rfm dataset:", missing_briefs_in_rfm)


Missing 'Brief' values in the client dataset: {'Sustainable Home prototype', 'Rebrand & Research', 'Equity Release campaign', 'Brand Escape Room Production', '3 Yr OneApp UX/UI Deal', 'Social Media support', 'Employer Value Proposition Creative Partner', 'Book design and production', 'Digital Marketing Campaign', 'FIFA website', 'Escape Travel eDM Build', 'Soho Blitz Day Media Support', 'Electrification Qual Research', 'Expert Videos x5 Bristol', 'UX Design System', 'Call centre automation', 'Sustainability Copywriting  - Extension', 'Shepper B2B Marketing', 'Project Trident', 'Dementia Proposition'}
Missing 'Brief' values in the rfm dataset: {'IMWT Scope Extension', 'Escape Room York', 'XD team support and capability build', 'SEO Aviva', '2+ Yr OneApp UX/UI Deal', 'Gen AI smoke Tests', 'MA Financial Website - US Version', 'Retirement Modelling Tool', 'Research help', 'MOCO 2023 - Agency fee', 'James Hardie PDF to Microsite Build', 'CBUS Claims Sprint', 'Panthera OneApp Ongoing UX/UI 

In [31]:
# Finding values in the 'Agency' column that are different between the two datasets
different_agencies = set(client['Agency']).symmetric_difference(set(rfm['Agency']))
if different_agencies:
    print("\nDifferences in 'Agency' column:")
    print(different_agencies)


Differences in 'Agency' column:
{'Ayo Banton', 'Deepend', 'Vivaldi', 'Ordinary Media', 'Energy Saving Trust', 'Studiomade', 'CS UX', 'Pavilion Works', 'Inflorescence Media', 'Snowmelt', 'Off Brand'}


In [32]:
# Finding values in the 'Agency' column that are present in one dataset but not the other
missing_agencies_in_client = set(client['Agency']) - set(rfm['Agency'])
missing_agencies_in_rfm = set(rfm['Agency']) - set(client['Agency'])
print("Missing 'Agency' values in the client dataset:", missing_agencies_in_client)
print("Missing 'Agency' values in the rfm dataset:", missing_agencies_in_rfm)


Missing 'Agency' values in the client dataset: {'CS UX', 'Ayo Banton', 'Ordinary Media', 'Energy Saving Trust', 'Studiomade', 'Pavilion Works', 'Off Brand'}
Missing 'Agency' values in the rfm dataset: {'Snowmelt', 'Vivaldi', 'Deepend', 'Inflorescence Media'}


In [33]:
unique_agencies_in_rfm = rfm['Agency'].unique()
print("Unique values in the rfm dataset's 'Agency' column:")
print(unique_agencies_in_rfm)

Unique values in the rfm dataset's 'Agency' column:
['Hedgehog Lab' 'Shoreditch Design Studio' 'GH05T Ltd.' 'Buddy Media'
 'bbp' 'Cocogun' 'Houston Group' 'Principals' 'Studiospace'
 'Notepad Studio' 'Orange Panther Collective' '2050London' 'NMD+' 'Apadmi'
 'xDesign' 'Reed Words' 'Viewpoint Studios' 'SAND visuals' 'Innovate Live'
 'S+Co' 'The Park' 'Good Innovation' 'UNPITCHD' 'Secret Source Marketing'
 'Croud' 'Inflorescence Media' 'CreateFuture' 'Loomery' 'Rip + Tear'
 'I-AM' 'Rufus Leonard' 'Heur' 'Three Whiskey' 'Nile HQ' 'Sparksfly'
 'Nimbletank' 'Neu' 'off brand.' 'G-W' 'Blicx' 'Frontier' 'HACKMASTERS'
 'Oxara London' 'Blue Feathers' 'Daughter Studio' 'Boost Awards' 'Inc'
 'InspoHub Ltd' 'Apex1' 'Stride' 'Salamandra.uk Animation Studios'
 'Tinker Taylor' 'Experience' 'Solution17' 'Vivaldi' 'Path Ventures'
 'Sketch & Co' 'VERSA AGENCY' 'OHMY' 'Solace Digital' 'Hatch Films CIC'
 'Josephmark' 'Studio Something' 'Akcelo' 'SLIK' 'Future Friendly'
 'Tobias' 'How To Impact' 'Slingshot' 

In [34]:
unique_agencies_in_client = client['Agency'].unique()
print("Unique values in the client dataset's 'Agency' column:")
print(unique_agencies_in_client)

Unique values in the client dataset's 'Agency' column:
['Boost Awards' 'Reed Words' 'Viewpoint Studios'
 'Shoreditch Design Studio' 'Blue Feathers' 'Daughter Studio'
 'CreateFuture' 'Oxara London' 'Secret Source Marketing' 'HACKMASTERS'
 'Loomery' 'Hedgehog Lab' 'GH05T Ltd.' 'Buddy Media' 'bbp' 'Cocogun'
 'Houston Group' 'Principals' 'Apadmi' 'xDesign' 'Good Innovation'
 'UNPITCHD' 'Innovate Live' 'Pavilion Works' 'Studio Something'
 'Orange Panther Collective' 'Nimbletank' 'NMD+' 'SAND visuals'
 'Energy Saving Trust' 'S+Co' 'The Park' 'G-W' 'I-AM' 'Blicx'
 'Notepad Studio' '2050London' 'Studiospace' 'Rip + Tear' 'Rufus Leonard'
 'Polar Media House' 'Frontier' 'Heur' 'Three Whiskey' 'Nile HQ'
 'Sparksfly' 'Neu' 'off brand.' 'Experience' 'Apex1' 'Solution17' 'Inc'
 'Ordinary Media' 'InspoHub Ltd' 'Salamandra.uk Animation Studios'
 'Tinker Taylor' 'Manifest Group' 'Sketch & Co' 'Ayo Banton' 'Stride'
 'Path Ventures' 'VERSA AGENCY' 'OHMY' 'Solace Digital' 'Hatch Films CIC'
 'Josephmark' '

In [35]:
unique_briefs_in_client = client['Brief'].unique()
print("Unique values in the client dataset's 'Brief' column:")
print(unique_briefs_in_client)

Unique values in the client dataset's 'Brief' column:
['Awards applications x5' 'Countrywide broker video'
 'Prop Dev (EV and CC Insurance)' 'Digital Claims'
 'Connected Car prop dev sprint' 'Senior UI Designer'
 'UX 2 month Resourcing' 'EV Location Partner Campaign' 'Brand Refresh'
 'Pre-retirement App' 'Protection proposition exploration'
 'DW Retained Squad' 'Choose Your Adventure' 'Expert Videos x5 Bristol'
 'Equity Release campaign' 'DW - Pension Consolidation journey design'
 'Generative AI' 'Escape Room Norwich' 'Buddy Media Aviva'
 'Dementia Proposition' 'Brand Escape Room Production' 'Climate  Report'
 'Innovation smoke tests' 'Escape Room Hoxton'
 'Aviva Masterclass - TikTok' 'DW Media Buying' 'DW Customer Strategy'
 'Aviva Award Entries' 'Aviva Social Media Team' 'SHIRO customer journeys'
 'One Aviva Video' 'V2 Aviva Advisor Prop Messaging' 'GI Retained Squad'
 'Expert Videos x5 London' 'Quote Me Happy Content'
 'Sustainable Home prototype' 'MOCO marketing Jul-Oct extension'

In [36]:
unique_briefs_in_rfm = rfm['Brief'].unique()
print("Unique values in the client dataset's 'Brief' column:")
print(unique_briefs_in_rfm)

Unique values in the client dataset's 'Brief' column:
['Senior UI Designer' 'UX 2 month Resourcing'
 'EV Location Partner Campaign' 'Brand Refresh' 'DW Overarching Vision'
 'Investor application journey' 'Quote Me Happy Content'
 'Group Life Content' 'DW E2E Prototype' 'DW Customer Strategy'
 'Pre-retirement App' 'Customer Engagement Triggers'
 'PRIS Video Production' 'Expert Videos x5 London' 'DW Media Buying'
 'Game of Risk' 'Brand Escape Room Design' 'Aviva Advisor Prop Messaging'
 'Expert object photography' 'V2 Aviva Advisor Prop Messaging'
 'Escape Room York' 'Escape Room Hoxton' 'Aviva Social Media Team'
 'Climate  Report' 'Aviva Award Entries' 'Generative AI'
 'Choose Your Adventure' 'One Aviva Video' 'Escape Room Norwich'
 'Protection proposition exploration' 'Adivsor Messaging'
 'Aviva Masterclass - TikTok' 'Buddy Media Aviva' 'SEO Aviva'
 'Gen AI smoke Tests' 'Zero micro-site' 'MoCo marketing'
 'MOCO Board Comms' 'MOCO marketing Jul-Oct extension'
 'MOCO Marketing (November/

In [37]:
rfm['Agency'] = rfm['Agency'].replace('Off Brand', 'off brand.')

### merging for client and rfm = client_rfm

In [38]:
# RFM + CLIENT
# Merge the dataframes on the specified columns
client_rfm = pd.merge(client, rfm, on=["Brief", "Agency"], how="inner")
client_rfm

,Brief,Client_x,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction,Client_y,Project extension \nwith same agency,...,Strong track record,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Match Score
0,Awards applications x5,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0,AXA,0,...,1,0,0,0,0,0,0,1,0,0.418605
1,Awards applications x5,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0,AXA,0,...,1,0,0,0,0,0,0,1,0,0.418605
2,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0,AXA,0,...,0,0,0,0,0,0,1,0,0,0.372093
3,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0,AXA,0,...,1,0,0,0,0,0,1,0,0,0.465116
4,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0,AXA,0,...,0,0,0,0,0,0,0,1,0,0.348837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Social Media Veezu,Veezu,Started,GH05T Ltd.,GH05T Ltd.,united kingdom,3,2.0,Veezu,0,...,1,0,0,0,0,1,0,0,0,0.604651
247,Social Media Veezu,Veezu,Started,Nonsensical,GH05T Ltd.,united kingdom,3,2.0,Veezu,0,...,0,0,0,0,0,1,0,0,0,0.139535
248,Social Media Veezu,Veezu,Started,Spin Brands,GH05T Ltd.,united kingdom,3,2.0,Veezu,0,...,1,0,0,0,0,1,0,0,0,0.325581
249,SME engagement,Wave,Archived,NMD+,NMD+,united kingdom,1,2.0,Wave,0,...,0,0,0,0,0,1,0,0,0,0.023256


In [39]:
client_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 26 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief                                               251 non-null    object 
 1   Client_x                                            251 non-null    object 
 2   Status                                              251 non-null    object 
 3   Agency                                              251 non-null    object 
 4   Winning Agency                                      251 non-null    object 
 5   Region                                              251 non-null    object 
 6   Agency Count                                        251 non-null    int64  
 7   Client_WA_Interaction                               251 non-null    float64
 8   Client_y                                            251 non-null    object 
 9  

In [40]:
# Check if the two columns are the same
differences = client_rfm[client_rfm['Client_x'] != client_rfm['Client_y']]

if differences.empty:
    print("The columns are identical. Dropping one of them.")
    client_rfm.drop(columns=['Client_y'], inplace=True)
else:
    print("The columns are not identical. Differences found:")
    print(differences[['Client_x', 'Client_y']])


The columns are not identical. Differences found:
               Client_x      Client_y
84                Aviva          Wave
85                Aviva          Wave
175  MA Financial Group  MA Financial
176  MA Financial Group  MA Financial
177  MA Financial Group  MA Financial
178  MA Financial Group  MA Financial
179  MA Financial Group  MA Financial
180  MA Financial Group  MA Financial
181  MA Financial Group  MA Financial
182  MA Financial Group  MA Financial
183  MA Financial Group  MA Financial
184  MA Financial Group  MA Financial
185  MA Financial Group  MA Financial


In [41]:
client_rfm.drop(columns=['Client_y'], inplace=True)
client_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief                                               251 non-null    object 
 1   Client_x                                            251 non-null    object 
 2   Status                                              251 non-null    object 
 3   Agency                                              251 non-null    object 
 4   Winning Agency                                      251 non-null    object 
 5   Region                                              251 non-null    object 
 6   Agency Count                                        251 non-null    int64  
 7   Client_WA_Interaction                               251 non-null    float64
 8   Project extension 
with same agency                 251 non-null    int64  
 9  

In [42]:
client_rfm.rename(columns={'Client_x': 'Client'}, inplace=True)

In [ ]:
#old code

In [ ]:
# # RFM + CLIENT
# # Merge the dataframes on the specified columns
# client_rfm = pd.merge(client, rfm, on=["Brief", "Agency", "Client"], how="inner")

In [53]:
# # RFM + CLIENT
# # Merge the dataframes on the specified columns
# client_rfm = pd.merge(client, rfm, on=["Brief", "Agency", "Client"], how="inner")

In [54]:
# client_rfm

,Brief,Client,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,Right geography\n / time zone,Strong skill\nset match,Deep / relevant\nsector expertise,Good cultural fit,Right price level,Strong track record,Match Score
0,awards applications x5,axa,started,boost awards,boost awards,united kingdom,2,1.0,0,0.0,1,1,0,0,0,1,0.418605
1,awards applications x5,axa,started,reed words,boost awards,united kingdom,2,1.0,0,0.0,1,1,0,0,0,1,0.418605
2,countrywide broker video,axa,archived,viewpoint studios,shoreditch design studio,united kingdom,2,1.0,0,0.0,0,1,0,0,1,0,0.372093
3,countrywide broker video,axa,archived,shoreditch design studio,shoreditch design studio,united kingdom,2,1.0,0,0.0,0,1,0,0,1,1,0.465116
4,prop dev ev cc insurance,axa,started,blue feathers,daughter studio,united kingdom,3,1.0,0,1.0,1,1,0,0,0,0,0.348837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,brand positioning,totalenergies,matching,nursery research planning ltd,unknown,united kingdom,4,1.0,0,0.0,1,1,1,0,1,1,0.790698
228,social media veezu,veezu,started,nonsensical,gh05t ltd .,united kingdom,3,2.0,0,0.0,0,0,0,0,1,0,0.139535
229,social media veezu,veezu,started,spin brands,gh05t ltd .,united kingdom,3,2.0,0,0.0,0,1,0,0,0,1,0.325581
230,sme engagement,wave,archived,nmd,nmd,united kingdom,1,2.0,0,1.0,0,0,0,0,0,0,0.023256


In [55]:
# client_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232 entries, 0 to 231
Data columns (total 17 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief                                               232 non-null    object 
 1   Client                                              232 non-null    object 
 2   Status                                              232 non-null    object 
 3   Agency                                              232 non-null    object 
 4   Winning Agency                                      232 non-null    object 
 5   Region                                              232 non-null    object 
 6   Agency Count                                        232 non-null    int64  
 7   Client_WA_Interaction                               232 non-null    float64
 8   Project extension 
with same agency                 232 non-null    int64  
 9  

### merging for client_rfm and agency = client_agency_rfm 

In [43]:
agency.head()

,Agency,Skills,Industries,Service Description,Vision,Industry Experience,Agency short description,Agency services,Any other services,Locations
0,5 Ways To Innovate,Growth & Innovation Strategy,"Financial Services,Technology, Information and...",We help corporates unlock the capability to re...,NaN,NaN,"We’re on a mission to help 1,000 corporates re...","Strategic Design, Leadership Development & Coa...","Innovation Strategy, Corporate Innovation",Australia
1,Arktic Fox,"Data Strategy, Digital Strategy, eCommerce & M...","Alcoholic Beverages, Education, Fashion, FMCG,...",Arktic Fox is a digital & eCommerce advisory a...,NaN,NaN,"We are on a mission to help marketing, digital...",CRM & Loyalty,"eCommerce strategy, Digital Transformation, Di...",Australia
2,Boost Awards,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,NaN,NaN,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,United Kingdom
3,Bread Agency,"Digital Strategy, Social Media, Creative Produ...","Alcoholic Beverages, Automotive, Beauty, Enter...",Social media agency making content people want...,NaN,NaN,Social media agency making content people want...,"Research & Insight, Creative & Production, Soc...","Influencer Marketing, Corporate Reputation, Co...",Australia
4,Chs,NaN,NaN,"Award-Winning, Digital First, Full Service Agency",NaN,NaN,"Award-Winning, Digital First, Full Service Agency","Brand Strategy, Creative & Production, Digital...",Photography and Film,"United Kingdom, United States"


In [44]:
client_rfm.head()

,Brief,Client,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,...,Strong track record,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Match Score
0,Awards applications x5,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0,0,0.0,...,1,0,0,0,0,0,0,1,0,0.418605
1,Awards applications x5,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0,0,0.0,...,1,0,0,0,0,0,0,1,0,0.418605
2,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0,0,0.0,...,0,0,0,0,0,0,1,0,0,0.372093
3,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0,0,0.0,...,1,0,0,0,0,0,1,0,0,0.465116
4,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0,0,1.0,...,0,0,0,0,0,0,0,1,0,0.348837


In [45]:
# Finding values in the 'Agency' column that are different between the two datasets
different_agencies = set(client_rfm['Agency']).symmetric_difference(set(agency['Agency']))
if different_agencies:
    print("\nDifferences in 'Agency' column:")
    print(different_agencies)


Differences in 'Agency' column:
{'Beesting Digital', 'Ayo Banton', 'Mkg Marketing', 'I-Am', 'SLIK', 'Midnight', 'Stakk', 'Honest Fox', 'Coredo', 'The Builders Arms Ltd', 'Deepend', 'Xdesign', 'Nzime', 'Ordinary Media', 'VERSA AGENCY', 'Nile HQ', 'Slik', 'IE', 'Hatch Films Cic', 'Nowadays', 'Sitback Solutions', 'HACKMASTERS', 'Nusa Films', 'Launch Studio', 'Allied Global Marketing', 'Energy Saving Trust', 'Arktic Fox', 'Studiomade', 'Taxi Studio', 'Synthetic', 'Versa Agency', 'Do. Agency', 'Media Bounty', 'Head & Heart Creative', 'Bump', 'Ubiquitous', 'Studio Blup', '5 Ways To Innovate', 'Submerge', 'William Joseph', 'Off Brand.', 'Comuzi', 'Five by Five Global', 'Pavilion Works', 'Harrison', 'Mymavins', 'blueegg', 'Unpitchd', 'Disrupt', 'Anchora', '44 Digital', 'PALO IT', 'Big Tent Strategy', 'Launchsquad', 'Rocketer', 'Communique', 'Reimagine Digital', 'InspoHub Ltd', 'NMD+', 'off brand.', 'Nibble Digital', 'Armadillo', 'Braidr Limited', 'Sand Visuals', 'The Curious Co', 'Five By Fiv

In [46]:
# Finding values in the 'Agency' column that are present in one dataset but not the other
missing_agencies_in_client_rfm = set(client_rfm['Agency']) - set(agency['Agency'])
missing_agencies_in_agency = set(agency['Agency']) - set(client_rfm['Agency'])
print("Missing 'Agency' values in the client_rfm dataset:", missing_agencies_in_client_rfm)
print("Missing 'Agency' values in the agency dataset:", missing_agencies_in_agency)

Missing 'Agency' values in the client_rfm dataset: {'I-AM', 'CreateFuture', 'Future Friendly', 'HeyBigMan!', 'SAND visuals', 'SLIK', 'Five by Five Global', 'GH05T Ltd.', 'blueegg', 'PALO IT', 'InspoHub Ltd', 'Hatch Films CIC', 'UNPITCHD', 'NMD+', 'Salamandra.uk Animation Studios', 'VERSA AGENCY', 'off brand.', 'Nile HQ', 'IE', 'Helium', 'bbp', 'HACKMASTERS', 'Launch Studio', '2CV', 'MyMavins', 'Slingshot', 'Quill Peak Consulting', 'Principals', 'xDesign', 'Rufus Leonard', 'Innovation Media', 'OHMY', 'Studiospace', 'Head & Heart Creative', 'Catch the Sun Communications', 'infogr8 Ltd', 'MB&a'}
Missing 'Agency' values in the agency dataset: {'Beesting Digital', 'Ayo Banton', 'Mkg Marketing', 'I-Am', 'Midnight', 'Stakk', 'Coredo', 'Honest Fox', 'The Builders Arms Ltd', 'Deepend', 'Xdesign', 'Nzime', 'Ordinary Media', 'Slik', 'Nowadays', 'Hatch Films Cic', 'Sitback Solutions', 'Nusa Films', 'Allied Global Marketing', 'Energy Saving Trust', 'Arktic Fox', 'Studiomade', 'Taxi Studio', 'Synthe

In [47]:
unique_agencies_in_client_rfm = client_rfm['Agency'].unique()
print("Unique values in the client_rfm dataset's 'Agency' column:")
print(unique_agencies_in_client_rfm)

Unique values in the client_rfm dataset's 'Agency' column:
['Boost Awards' 'Reed Words' 'Viewpoint Studios'
 'Shoreditch Design Studio' 'Blue Feathers' 'Daughter Studio'
 'CreateFuture' 'Oxara London' 'Secret Source Marketing' 'HACKMASTERS'
 'Loomery' 'Hedgehog Lab' 'GH05T Ltd.' 'Buddy Media' 'bbp' 'Cocogun'
 'Houston Group' 'Principals' 'Apadmi' 'xDesign' 'Good Innovation'
 'UNPITCHD' 'Innovate Live' 'Nimbletank' 'NMD+' 'SAND visuals' 'S+Co'
 'The Park' 'G-W' 'I-AM' 'Blicx' 'Notepad Studio'
 'Orange Panther Collective' '2050London' 'Studiospace' 'Rip + Tear'
 'Rufus Leonard' 'Polar Media House' 'Frontier' 'Heur' 'Three Whiskey'
 'Nile HQ' 'Sparksfly' 'Neu' 'off brand.' 'Experience' 'Apex1'
 'Solution17' 'Inc' 'InspoHub Ltd' 'Salamandra.uk Animation Studios'
 'Tinker Taylor' 'Path Ventures' 'VERSA AGENCY' 'Sketch & Co' 'OHMY'
 'Solace Digital' 'Hatch Films CIC' 'Josephmark' 'Croud'
 'Studio Something' 'Akcelo' 'SLIK' 'Future Friendly' 'Tobias'
 'How To Impact' 'Slingshot' 'Innovation M

In [48]:
unique_agencies_in_agency = agency['Agency'].unique()
print("Unique values in the agency dataset's 'Agency' column:")
print(unique_agencies_in_agency)

Unique values in the agency dataset's 'Agency' column:
['5 Ways To Innovate' 'Arktic Fox' 'Boost Awards' 'Bread Agency' 'Chs'
 'Croud' 'Cxr Agency' 'Edge' 'Experience' 'Five By Five Global'
 'Hidden Strategy' 'Honest Fox' 'Launchsquad' 'Midnight'
 'North Street Creative' 'Rebel Angel Strategy' 'Rehab' 'Rocketer'
 'Screentide' 'Something™' 'Study Hall' 'The Curious Co'
 'The Nursery Research & Planning Ltd' 'Ubiquitous' 'Flume' 'Mymavins'
 '2Cv' 'Studio Blup' 'Remesh' 'Phase One Insights' 'Hippo Digital'
 'Infogr8 Ltd' 'Alignalytics' 'Blicx' 'Blueegg' 'Objective Experience'
 'Verve' 'Nmd+' 'Cyber-Duck' '44 Digital' 'Disrupt'
 'Catch The Sun Communications' 'Beesting Digital' 'Airview Productions'
 'Hatch Films Cic' 'Nowadays' 'Studio Yes' 'Inspohub Ltd' 'Bearjam'
 'Nusa Films' 'Tinker Taylor' 'Videomachine Limited' 'Viewpoint Studios'
 'Fun Agency Ltd' 'Reed Words' 'Shoreditch Design Studio'
 'Studio Something' 'The Builders Arms Ltd' 'Studiomade' 'Common Ground'
 '2050London' 'Taxi Stu

In [49]:
agency['Agency'] = agency['Agency'].str.strip()
client_rfm['Agency'] = client_rfm["Agency"].str.strip()

In [50]:
replacements = {
    'xDesign': 'xDesign',
    'Createfuture' : 'CreateFuture', 
    'Hackmasters' : 'HACKMASTERS', 
    'Unpitchd' : 'UNPITCHD',
    'Sand Visuals': 'SAND visuals',
    'Rip + Tear' : 'Rip + Tear',
    'Versa Agency' : 'VERSA AGENCY',
    'Ohmy' : 'OHMY',
    'Mb&A' : 'MB&a',
    'Ie' : 'IE',
    'Infogr8 Ltd' : 'infogr8 Ltd',
    'Catch The Sun Communications': 'Catch the Sun Communications',
    'Mymavins': 'MyMavins'
}
agency['Agency'] = agency['Agency'].str.strip().replace(replacements)


In [51]:
# Mapping of incorrect values to correct values
fix_mapping = {
    '5 Ways to Innovate': '5 Ways To Innovate',
    'Salamandra.Uk Animation Studios': 'Salamandra.uk Animation Studios',
    'Bbp': 'bbp',
    'Gh05T Ltd.': 'GH05T Ltd.',
    'I-Am': 'I-AM',
    'Off Brand.': 'off brand.',
    'Heybigman!': 'HeyBigMan!',
    '2Cv': '2CV',
    'Slik': 'SLIK',
    'Five By Five Global': 'Five by Five Global',
    'Blueegg': 'blueegg',
    'Inspohub Ltd': 'InspoHub Ltd',
    'Palo It': 'PALO IT',
    'Hatch Films Cic': 'Hatch Films CIC',
    'Xdesign': 'xDesign'
}

# Apply the mapping to fix the values in the agency dataset
agency['Agency'] = agency['Agency'].replace(fix_mapping)

# Apply the mapping to fix the values in the client_rfm dataset
client_rfm['Agency'] = client_rfm['Agency'].replace(fix_mapping)


In [52]:
# Merge the dataframes on the 'Agency' column
client_agency_rfm = pd.merge(client_rfm, agency, on='Agency', how='left')
client_agency_rfm

,Brief,Client,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,...,Match Score,Skills,Industries,Service Description,Vision,Industry Experience,Agency short description,Agency services,Any other services,Locations
0,Awards applications x5,AXA,Started,Boost Awards,Boost Awards,united kingdom,2,1.0,0,0.0,...,0.418605,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,NaN,NaN,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,United Kingdom
1,Awards applications x5,AXA,Started,Reed Words,Boost Awards,united kingdom,2,1.0,0,0.0,...,0.418605,"Brand Strategy, Copywriting, Corporate Communi...","Administrative and Support Services, Alcoholic...",Verbal branding and copy writing,Making Brands and Businesses Stronger Through ...,"FMCG,Retail,Non-Profit",We make brands and business stronger through l...,"Marketing Planning, Social & Content","Copywriting, Naming, Tone of Voice, Brand Voic...","United Kingdom, United States"
2,Countrywide broker video,AXA,Archived,Viewpoint Studios,Shoreditch Design Studio,united kingdom,2,1.0,0,0.0,...,0.372093,"Brand Design, Customer Experience Design, Grap...","Alcoholic Beverages, Automotive, Beauty, Const...",Advertising and commercial photography and film,NaN,NaN,Inspired photography and film,"Creative & Production, Social & Content",Advertising and commercial photography and film,United Kingdom
3,Countrywide broker video,AXA,Archived,Shoreditch Design Studio,Shoreditch Design Studio,united kingdom,2,1.0,0,0.0,...,0.465116,"Brand Design, UX & UI Design, Mobile App Devel...",B2B,The creative agency for your next big thing,We create high quality digital design for busi...,"Automotive,Fitness,Health,Insurance,Fintech,Te...",The creative agency for your next big thing,"Strategic Design, UX/UI Design","Product Design, Print Design, Animation, Illus...","Australia, United Kingdom, United States"
4,Prop Dev (EV and CC Insurance),AXA,Started,Blue Feathers,Daughter Studio,united kingdom,3,1.0,0,1.0,...,0.348837,"Business Strategy, CRM Strategy, Customer Stra...","Alcoholic Beverages, Beauty, Education, Entert...",We work with ambitious marketing leaders and t...,Specialists in brand strategy and proposition ...,"Insurance,Financial Services,Waste Management,...",We work with ambitious marketing leaders and t...,"Brand Strategy, Proposition Design, CRM & Loyalty",NaN,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Social Media Veezu,Veezu,Started,GH05T Ltd.,GH05T Ltd.,united kingdom,3,2.0,0,0.0,...,0.604651,"Data Analytics, Advertising, Brand Strategy, S...","Alcoholic Beverages, Beauty, Entertainment, Fa...",We are a social media agency operating in the ...,NaN,NaN,We are a social media agency operating in the ...,"Brand Strategy, Creative & Production, Marketi...",NaN,"United Kingdom, United States"
247,Social Media Veezu,Veezu,Started,Nonsensical,GH05T Ltd.,united kingdom,3,2.0,0,0.0,...,0.139535,"Advertising, Social Media, Creative Production...","Beauty, Education, Entertainment, Fashion, Fin...",We do TikTok so you don't have to!,NaN,NaN,We do TikTok so you don't have to!,"Creative & Production, Social & Content, Digit...",TikTok,United Kingdom
248,Social Media Veezu,Veezu,Started,Spin Brands,GH05T Ltd.,united kingdom,3,2.0,0,0.0,...,0.325581,"Digital Strategy, Growth & Innovation Strategy...","Administrative and Support Services, Alcoholic...",Voted UK's #1 Social Agency,NaN,NaN,Voted UK's #1 Social Agency,"Creative & Production, Marketing Planning, Soc...","Social Media Management, Advertising + Content",United Kingdom
249,SME engagement,Wave,Archived,NMD+,NMD+,united kingdom,1,2.0,0,1.0,...,0.023256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
client_agency_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 34 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief                                               251 non-null    object 
 1   Client                                              251 non-null    object 
 2   Status                                              251 non-null    object 
 3   Agency                                              251 non-null    object 
 4   Winning Agency                                      251 non-null    object 
 5   Region                                              251 non-null    object 
 6   Agency Count                                        251 non-null    int64  
 7   Client_WA_Interaction                               251 non-null    float64
 8   Project extension 
with same agency                 251 non-null    int64  
 9  

## MERGED dataset: MERGE.csv 
### contains 251 instances

In [118]:
#old code

In [57]:
# # Preprocess the 'Agency' column in both dataframes as done before
# agency['Agency'] = agency['Agency'].str.strip().str.lower().fillna('').str.replace('[^\w\s]', '')
# client_rfm['Agency'] = client_rfm['Agency'].str.strip().str.lower().fillna('').str.replace('[^\w\s]', '')

# # Merge the dataframes on the 'Agency' column
# client_agency_rfm = pd.merge(client_rfm, agency, on='Agency', how='left')
# client_agency_rfm

/tmp/ipykernel_230/845558983.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  agency['Agency'] = agency['Agency'].str.strip().str.lower().fillna('').str.replace('[^\w\s]', '')
/tmp/ipykernel_230/845558983.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  client_rfm['Agency'] = client_rfm['Agency'].str.strip().str.lower().fillna('').str.replace('[^\w\s]', '')


,Brief,Client,Status,Agency,Winning Agency,Region,Agency Count,Client_WA_Interaction,Project extension \nwith same agency,Client selected agency\nthemselves from marketplace,...,Right price level,Strong track record,Match Score,Skills,Industries,Service Description,Agency short description,Agency services,Any other services,Locations
0,awards applications x5,axa,started,boost awards,boost awards,united kingdom,2,1.0,0,0.0,...,0,1,0.418605,"copywriting , pr",professional services,world ’ first largest award entry consultancy,world ’ first largest award entry consultancy,public relation,award consultancy,united kingdom
1,awards applications x5,axa,started,reed words,boost awards,united kingdom,2,1.0,0,0.0,...,0,1,0.418605,"brand strategy , copywriting , corporate commu...","administrative support services , alcoholic be...",verbal branding copy writing,make brand business stronger language,marketing planning social content,copywriting naming tone voice brand voice verb...,"united kingdom , united states"
2,countrywide broker video,axa,archived,viewpoint studios,shoreditch design studio,united kingdom,2,1.0,0,0.0,...,1,0,0.372093,"brand design , customer experience design , gr...","alcoholic beverages , automotive , beauty , co...",advertising commercial photography film,inspired photography film,creative production social content,advertising commercial photography film,united kingdom
3,countrywide broker video,axa,archived,shoreditch design studio,shoreditch design studio,united kingdom,2,1.0,0,0.0,...,1,1,0.465116,"brand design , ux ui design , mobile app devel...",b2b,creative agency next big thing,creative agency next big thing,strategic design uxui design,product design print design animation illustra...,"australia , united kingdom , united states"
4,prop dev ev cc insurance,axa,started,blue feathers,daughter studio,united kingdom,3,1.0,0,1.0,...,0,0,0.348837,"business strategy , crm strategy , customer st...","alcoholic beverages , beauty , education , ent...",work ambitious marketing leader team help navi...,work ambitious marketing leader team help navi...,brand strategy proposition design crm loyalty,NaN,united kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,brand positioning,totalenergies,matching,nursery research planning ltd,unknown,united kingdom,4,1.0,0,0.0,...,1,1,0.790698,"social media , video production",b2b,specialist brand communication experience rese...,specialist brand communication experience rese...,research insight,NaN,united kingdom
228,social media veezu,veezu,started,nonsensical,gh05t ltd .,united kingdom,3,2.0,0,0.0,...,1,0,0.139535,"advertising , social media , creative producti...","beauty , education , entertainment , fashion ,...",tiktok dont,tiktok dont,creative production social content digital mar...,tiktok,united kingdom
229,social media veezu,veezu,started,spin brands,gh05t ltd .,united kingdom,3,2.0,0,0.0,...,0,1,0.325581,"digital strategy , growth innovation strategy ...","administrative support services , alcoholic be...",voted uk social agency,voted uk social agency,creative production marketing planning social ...,social medium management advertising content,united kingdom
230,sme engagement,wave,archived,nmd,nmd,united kingdom,1,2.0,0,1.0,...,0,0,0.023256,"business strategy , customer strategy , digita...",financial services,creative technology consultancy,specialist agency focused helping company buil...,uxui design strategic design social content,specalists financial service industry,united kingdom


In [86]:
# client_agency_rfm.info()

In [54]:
#renaming columns 
client_agency_rfm = client_agency_rfm.rename(columns={"Brief": "Brief Name", 
                                                      "Category": "Brief Category", 
                                                      "Agency": "Shortlisted Agency",
                                                     "Region": "Brief Region",
                                                     "Agency Count": "Number of Agencies Shortlisted Brief",
                                                     "Tags":"Brief Tags",
                                                     "Skills": "Agency Skills",
                                                     "Industries":"Agency Industries",
                                                     "Agency short description": "Agency Description",
                                                     "Any other services": "Other Services",
                                                     "Locations":"Agency Locations",
                                                     "Project extension \nwith same agency": "Project extension with same agency",
                                                      "Client selected agency\nthemselves from marketplace": "Client selected agency themselves from marketplace",
                                                      'Right geography\n / time zone': 'Right geography / time zone',
                                                      'Strong skill\nset match': 'Strong skill set match',
                                                      'Deep / relevant\nsector expertise': 'Deep / relevant sector expertise'})

In [55]:
client_agency_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 34 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client                                              251 non-null    object 
 2   Status                                              251 non-null    object 
 3   Shortlisted Agency                                  251 non-null    object 
 4   Winning Agency                                      251 non-null    object 
 5   Brief Region                                        251 non-null    object 
 6   Number of Agencies Shortlisted Brief                251 non-null    int64  
 7   Client_WA_Interaction                               251 non-null    float64
 8   Project extension with same agency                  251 non-null    int64  
 9  

In [56]:
desired_column_order = [
    'Brief Name',
    'Client',
    'Shortlisted Agency',
    'Agency Skills',
    'Agency Industries',
    'Service Description',
    'Agency Description',
    'Agency services',
    'Other Services',
    'Vision',
    'Industry Experience',
    'Agency Locations',
    'Brief Region',
    'Status',
    'Winning Agency',
    'Number of Agencies Shortlisted Brief',
    'Client_WA_Interaction',
    'Project extension with same agency',
    'Client selected agency themselves from marketplace',
    'Right geography / time zone',
    'Strong skill set match',
    'Deep / relevant sector expertise',
    'Good cultural fit',
    'Right price level',
    'Strong track record',
    'Match Score',
    'Lead_George Patten',
    'Lead_Gideon Hyde',
    'Lead_Hannah Fraser',
    'Lead_Kate Walker',
    'Lead_Paul Bowman',
    'Lead_Peter Sayburn',
    'Lead_Phil Kohler',
    'Lead_Robin Scarborough'
]

client_agency_rfm = client_agency_rfm[desired_column_order]

In [57]:
client_agency_rfm.head()

,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Vision,...,Strong track record,Match Score,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough
0,Awards applications x5,AXA,Boost Awards,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,NaN,...,1,0.418605,0,0,0,0,0,0,1,0
1,Awards applications x5,AXA,Reed Words,"Brand Strategy, Copywriting, Corporate Communi...","Administrative and Support Services, Alcoholic...",Verbal branding and copy writing,We make brands and business stronger through l...,"Marketing Planning, Social & Content","Copywriting, Naming, Tone of Voice, Brand Voic...",Making Brands and Businesses Stronger Through ...,...,1,0.418605,0,0,0,0,0,0,1,0
2,Countrywide broker video,AXA,Viewpoint Studios,"Brand Design, Customer Experience Design, Grap...","Alcoholic Beverages, Automotive, Beauty, Const...",Advertising and commercial photography and film,Inspired photography and film,"Creative & Production, Social & Content",Advertising and commercial photography and film,NaN,...,0,0.372093,0,0,0,0,0,1,0,0
3,Countrywide broker video,AXA,Shoreditch Design Studio,"Brand Design, UX & UI Design, Mobile App Devel...",B2B,The creative agency for your next big thing,The creative agency for your next big thing,"Strategic Design, UX/UI Design","Product Design, Print Design, Animation, Illus...",We create high quality digital design for busi...,...,1,0.465116,0,0,0,0,0,1,0,0
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,"Business Strategy, CRM Strategy, Customer Stra...","Alcoholic Beverages, Beauty, Education, Entert...",We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"Brand Strategy, Proposition Design, CRM & Loyalty",NaN,Specialists in brand strategy and proposition ...,...,0,0.348837,0,0,0,0,0,0,1,0


In [58]:
client_agency_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 34 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client                                              251 non-null    object 
 2   Shortlisted Agency                                  251 non-null    object 
 3   Agency Skills                                       210 non-null    object 
 4   Agency Industries                                   210 non-null    object 
 5   Service Description                                 220 non-null    object 
 6   Agency Description                                  226 non-null    object 
 7   Agency services                                     226 non-null    object 
 8   Other Services                                      155 non-null    object 
 9  

# ADDING THE NEW BRIEF TAXONOMY RECEIVED FROM STUDIOSPACE

In [59]:
new_tax = pd.read_csv("/project/NEW_TAGS.csv")
new_tax

,Brief Name,Sub Taxonomy,Main Taxonomy,Client,sentence
0,Countrywide broker video,(Video_Production),"(Production Services), (Advertising_Brand & Cr...",AXA,"The project, titled 'Countrywide broker video'..."
1,Awards applications x5,"(PR), (Corporate_Communication)",(Media_Public Relations & Events),AXA,"The project, titled 'Awards applications x5', ..."
2,Prop Dev (EV and CC Insurance),(Proposition_Design),(Experience & Design),AXA,"The project, titled 'Prop Dev (EV and CC Insur..."
3,Digital Claims,"(Customer_Experience_Design), (Copywriting), (...","(Experience & Design), (Advertising_Brand & Cr...",AXA,"The project, titled 'Digital Claims', is prima..."
4,Connected Car prop dev sprint,(Proposition_Design),(Experience & Design),AXA,"The project, titled 'Connected Car prop dev sp..."
...,...,...,...,...,...
204,Exec Workshop,(Leadership_&_Development),(Employer Brand & Experience),UBS,"The project, titled 'Exec Workshop', is primar..."
205,Social Media Veezu,(Social_Media_and_Influencer_Marketing),(Media_Public Relations & Events),Veezu,"The project, titled 'Social Media Veezu', is p..."
206,Social Media support,"(Social_Media), (Creative_Production)","(Media_Public Relations & Events), (Advertisin...",Veezu,"The project, titled 'Social Media support', is..."
207,SME engagement,(Proposition_Design),(Experience & Design),Wave,"The project, titled 'SME engagement', is prima..."


In [60]:
#creating a copy for ease of usage
df = client_agency_rfm.copy()

In [61]:
missing_in_df = set(new_tax['Brief Name']) - set(df['Brief Name'])
missing_in_new_tax = set(df['Brief Name']) - set(new_tax['Brief Name'])

print("Missing 'Brief Name' values in df:", missing_in_df)
print("Missing 'Brief Name' values in new_tax:", missing_in_new_tax)


Missing 'Brief Name' values in df: {'Aviva Mobility Brief', 'Pulse Project - Follow-on Prototype Design', 'Gen AI smoke Tests', 'Sustainable Home prototype', 'Biffa Corporate Event Support Phase 1 Design', 'User Experience/User Interface Retained Squads', 'Employer Value Proposition Creative Partner', 'Onboarding and servicing portal', 'Public Relations agency', 'Website & App', 'Electrification Qual Research', 'Vehicle Connected Services - Personas', 'UX partner - retainer', 'Search Engine Optimization - Legal Services', 'Cover Sense ph2 (UX)', 'UX Design Squad(s)', 'Acuity Magazine CA Audio Content Editing', 'OneApp UX Research Strategic Support', 'CBUS Claims Sprint', 'Panthera Research Partner', 'Aviva Funnel Conversion', 'Shepper B2B Marketing', 'Web & App redesign', 'Pulse Project - Phase 2 Research', 'HSBC Financial Education', 'New dairy food brand', 'HSBC InHouse Social Media', 'Research platform', 'My Aviva Social Content & Media', 'Brand Refresh - case for change (Consulting

In [62]:
# Display unique values for manual inspection
print("Unique values in df's 'brief name' column:", df['Brief Name'].unique())
print("Unique values in new_tax's 'brief name' column:", new_tax['Brief Name'].unique())

Unique values in df's 'brief name' column: ['Awards applications x5' 'Countrywide broker video'
 'Prop Dev (EV and CC Insurance)' 'Digital Claims'
 'Connected Car prop dev sprint' 'Senior UI Designer'
 'UX 2 month Resourcing' 'EV Location Partner Campaign' 'Brand Refresh'
 'Pre-retirement App' 'Protection proposition exploration'
 'DW Retained Squad' 'Choose Your Adventure'
 'DW - Pension Consolidation journey design' 'Generative AI'
 'Escape Room Norwich' 'Buddy Media Aviva' 'Climate  Report'
 'Innovation smoke tests' 'Escape Room Hoxton'
 'Aviva Masterclass - TikTok' 'DW Media Buying' 'DW Customer Strategy'
 'Aviva Award Entries' 'Aviva Social Media Team' 'SHIRO customer journeys'
 'One Aviva Video' 'V2 Aviva Advisor Prop Messaging' 'GI Retained Squad'
 'Expert Videos x5 London' 'Quote Me Happy Content'
 'MOCO marketing Jul-Oct extension' 'DW E2E Prototype' 'SHIRO follow on'
 'Aviva Advisor Prop Messaging' 'Brand Escape Room Design' 'MOCO content'
 'PRIS Video Production' 'Life QtB j

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 34 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client                                              251 non-null    object 
 2   Shortlisted Agency                                  251 non-null    object 
 3   Agency Skills                                       210 non-null    object 
 4   Agency Industries                                   210 non-null    object 
 5   Service Description                                 220 non-null    object 
 6   Agency Description                                  226 non-null    object 
 7   Agency services                                     226 non-null    object 
 8   Other Services                                      155 non-null    object 
 9  

In [64]:
# Merge the  new_tax DataFrame with the df DataFrame
MERGED = pd.merge(df, new_tax, on='Brief Name', how='left')

In [65]:
MERGED.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 38 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client_x                                            251 non-null    object 
 2   Shortlisted Agency                                  251 non-null    object 
 3   Agency Skills                                       210 non-null    object 
 4   Agency Industries                                   210 non-null    object 
 5   Service Description                                 220 non-null    object 
 6   Agency Description                                  226 non-null    object 
 7   Agency services                                     226 non-null    object 
 8   Other Services                                      155 non-null    object 
 9  

In [66]:
MERGED.drop(columns=['Client_y', 'sentence'], inplace=True)
MERGED.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251 entries, 0 to 250
Data columns (total 36 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client_x                                            251 non-null    object 
 2   Shortlisted Agency                                  251 non-null    object 
 3   Agency Skills                                       210 non-null    object 
 4   Agency Industries                                   210 non-null    object 
 5   Service Description                                 220 non-null    object 
 6   Agency Description                                  226 non-null    object 
 7   Agency services                                     226 non-null    object 
 8   Other Services                                      155 non-null    object 
 9  

In [67]:
# Renaming columns
MERGED.rename(columns={'Client_x': 'Client',
                       'Sub Taxonomy': 'Sub Brief Taxonomy',
                       'Main Taxonomy': 'Main Brief Taxonomy'}, inplace=True)

# Missing Data

## Brief Taxonomy Columns

In [68]:
missing_values = MERGED.isna().sum()
sentence = "Number of missing values per column:\n" + "\n".join([f"{col}: {missing}" for col, missing in missing_values.items() if missing > 0])
print(sentence)

Number of missing values per column:
Agency Skills: 41
Agency Industries: 41
Service Description: 31
Agency Description: 25
Agency services: 25
Other Services: 96
Vision: 127
Industry Experience: 130
Agency Locations: 25
Sub Brief Taxonomy: 17
Main Brief Taxonomy: 17


In [69]:
# treating for NaNs within the Brief Taxonomy
subset = MERGED.loc[MERGED['Main Brief Taxonomy'].isna() | MERGED['Sub Brief Taxonomy'].isna(), ['Brief Name', 
                                                                                                 'Client',
                                                                                                 'Shortlisted Agency',
                                                                                                 'Main Brief Taxonomy', 
                                                                                                 'Sub Brief Taxonomy']]
subset

,Brief Name,Client,Shortlisted Agency,Main Brief Taxonomy,Sub Brief Taxonomy
17,Brand Refresh,Auspaynet,Cocogun,NaN,NaN
18,Brand Refresh,Auspaynet,Houston Group,NaN,NaN
19,Brand Refresh,Auspaynet,Principals,NaN,NaN
34,Climate Report,Aviva,Reed Words,NaN,NaN
99,Biffa Corporate Event Support,Biffa,Innovate Live,NaN,NaN
100,Biffa Corporate Event Support,Biffa,Experience,NaN,NaN
101,Biffa Corporate Event Support,Biffa,The Park,NaN,NaN
130,SEO - Legal Services,Fletchers Solicitors,Three Whiskey,NaN,NaN
131,SEO - Legal Services,Fletchers Solicitors,Croud,NaN,NaN
151,HSBC Creative Resource,HSBC,GH05T Ltd.,NaN,NaN


In [70]:
# Treating for NaNs in the Sub Taxonomy column header
sub_brief_taxonomy_fill_values = {
    17: '(Brand_Strategy),(Brand_Design)',
    18: '(Brand_Strategy),(Brand_Design)',
    19: '(Brand_Strategy),(Brand_Design)',
    34: '(Copywriting)' ,
    99: '(Spatial_Design), (Events)',
    100: '(Spatial_Design), (Events)' ,
    101: '(Spatial_Design), (Events)',
    130: '(Search_Engine_Optimization/Search_Engine_Marketing), (Digital_Performance_Marketing), (PR)',
    131: '(Search_Engine_Optimization/Search_Engine_Marketing), (Digital_Performance_Marketing), (PR)',
    158: '(User_Experience/User_Interface_Design)',
    166: '(User_Experience/User_Interface_Design)',
    167: '(User_Experience/User_Interface_Design)',
    168: '(User_Experience/User_Interface_Design)',
    179: '(Search_Engine_Optimization/Search_Engine_Marketing), (Digital_Performance_Marketing)',
    180: '(Search_Engine_Optimization/Search_Engine_Marketing), (Digital_Performance_Marketing)',
    181: '(Search_Engine_Optimization/Search_Engine_Marketing), (Digital_Performance_Marketing)'
}
# Iterate through the dictionary and fill the values at the specified indices
for index, value in sub_brief_taxonomy_fill_values.items():
    MERGED.loc[index, 'Sub Brief Taxonomy'] = value

In [71]:
# Treating for NaNs i the Main Taxonomy column header
main_brief_taxonomy_fill_values = {
    17: '(Experience & Design), (Advertising_Brand & Creative)',
    18: '(Experience & Design), (Advertising_Brand & Creative)',
    19: '(Experience & Design), (Advertising_Brand & Creative)',
    34: '(Advertising_Brand & Creative)' ,
    99: '(Experience & Design), (Media_Public Relations & Events) ', 
    100: '(Experience & Design), (Media_Public Relations & Events) ' ,
    101: '(Experience & Design), (Media_Public Relations & Events) ',
    130: '(Media_Public Relations & Events)',
    131: '(Media_Public Relations & Events)',
    158: '(Experience & Design)',
    166:'(Experience & Design)',
    167: '(Experience & Design)',
    168: '(Experience & Design)',
    179: '(Media_Public Relations & Events)',
    180: '(Media_Public Relations & Events)',
    181: '(Media_Public Relations & Events)'
}

# Iterate through the dictionary and fill the values at the specified indices
for index, value in sub_brief_taxonomy_fill_values.items():
    MERGED.loc[index, 'Main Brief Taxonomy'] = value

In [72]:
missing_values = MERGED.isna().sum()
sentence = "Number of missing values per column:\n" + "\n".join([f"{col}: {missing}" for col, missing in missing_values.items() if missing > 0])
print(sentence)

Number of missing values per column:
Agency Skills: 41
Agency Industries: 41
Service Description: 31
Agency Description: 25
Agency services: 25
Other Services: 96
Vision: 127
Industry Experience: 130
Agency Locations: 25
Sub Brief Taxonomy: 1
Main Brief Taxonomy: 1


## Agency Location

In [73]:
subset_location = MERGED.loc[MERGED['Agency Locations'].isna(), 
                    ['Shortlisted Agency', 'Agency Locations']]
subset_location

,Shortlisted Agency,Agency Locations
19,Principals,NaN
29,NMD+,NaN
52,NMD+,NaN
61,NMD+,NaN
66,NMD+,NaN
74,Studiospace,NaN
82,Rufus Leonard,NaN
84,NMD+,NaN
94,Nile HQ,NaN
105,NMD+,NaN


In [74]:
agency_locations = {
    'NMD+': 'United Kingdom'
}

for agency_name, location_value in agency_locations.items():
    MERGED.loc[(MERGED['Agency Locations'].isna()) & (MERGED['Shortlisted Agency'].str.strip() == agency_name), 'Agency Locations'] = location_value


In [75]:
subset_location = MERGED.loc[MERGED['Agency Locations'].isna(), 
                    ['Shortlisted Agency', 'Agency Locations']]
subset_location

,Shortlisted Agency,Agency Locations
19,Principals,NaN
74,Studiospace,NaN
82,Rufus Leonard,NaN
94,Nile HQ,NaN
107,Rufus Leonard,NaN
142,Future Friendly,NaN
145,Slingshot,NaN
146,Innovation Media,NaN
191,Helium,NaN
215,Rufus Leonard,NaN


# treating for skills, industries

In [77]:
# Creating a subset for the specified columns where any of them contain NaNs
subset_skills_industries = MERGED.loc[
    MERGED['Agency Skills'].isna() |
    MERGED['Agency Industries'].isna(),
    [
        'Brief Name',
        'Client',
        'Shortlisted Agency',
        'Agency Skills',
        'Agency Industries'
    ]
]

# Display the subset
subset_skills_industries

,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries
12,Senior UI Designer,Aegon,Hedgehog Lab,NaN,NaN
17,Brand Refresh,Auspaynet,Cocogun,NaN,NaN
19,Brand Refresh,Auspaynet,Principals,NaN,NaN
25,DW Retained Squad,Aviva,Hedgehog Lab,NaN,NaN
29,DW - Pension Consolidation journey design,Aviva,NMD+,NaN,NaN
46,GI Retained Squad,Aviva,Hedgehog Lab,NaN,NaN
52,DW E2E Prototype,Aviva,NMD+,NaN,NaN
61,Life QtB journey optimisation,Aviva,NMD+,NaN,NaN
66,DW Personas,Aviva,NMD+,NaN,NaN
74,DW Overarching Vision,Aviva,Studiospace,NaN,NaN


In [78]:
subset_skills_industries["Shortlisted Agency"].value_counts()

NMD+                     9
Hedgehog Lab             5
Quill Peak Consulting    3
Rip + Tear               3
Rufus Leonard            3
Helium                   2
Sketch & Co              2
Cocogun                  2
blueegg                  1
MyMavins                 1
Launch Studio            1
Bridge Studio            1
Future Friendly          1
Innovation Media         1
Slingshot                1
Experience               1
Nile HQ                  1
Studiospace              1
Principals               1
Head & Heart Creative    1
Name: Shortlisted Agency, dtype: int64

## Industries and Vision

In [79]:
# Creating a subset for the specified columns where any of them contain NaNs
subset = MERGED.loc[
    MERGED['Vision'].isna() |
    MERGED['Industry Experience'].isna(),
    [
        'Brief Name',
        'Client',
        'Shortlisted Agency',
        'Vision',
        'Industry Experience'
    ]
]

# Display the subset
subset

,Brief Name,Client,Shortlisted Agency,Vision,Industry Experience
0,Awards applications x5,AXA,Boost Awards,NaN,NaN
2,Countrywide broker video,AXA,Viewpoint Studios,NaN,NaN
6,Prop Dev (EV and CC Insurance),AXA,CreateFuture,NaN,NaN
7,Digital Claims,AXA,Oxara London,NaN,NaN
8,Digital Claims,AXA,Secret Source Marketing,NaN,NaN
...,...,...,...,...,...
246,Social Media Veezu,Veezu,GH05T Ltd.,NaN,NaN
247,Social Media Veezu,Veezu,Nonsensical,NaN,NaN
248,Social Media Veezu,Veezu,Spin Brands,NaN,NaN
249,SME engagement,Wave,NMD+,NaN,NaN


## Service Description, Agency Description, Agency services, Other services

In [80]:
# Creating a subset for the specified columns where any of them contain NaNs
subset = MERGED.loc[
    MERGED['Service Description'].isna() |
    MERGED['Agency Description'].isna() |
    MERGED['Agency services'].isna() |
    MERGED['Other Services'].isna(),
    [
        'Brief Name',
        'Client',
        'Shortlisted Agency',
        'Service Description',
        'Agency Description',
        'Agency services',
        'Other Services'
    ]
]

# Display the subset
subset

,Brief Name,Client,Shortlisted Agency,Service Description,Agency Description,Agency services,Other Services
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"Brand Strategy, Proposition Design, CRM & Loyalty",NaN
5,Prop Dev (EV and CC Insurance),AXA,Daughter Studio,Independent creative agency of senior thinkers...,Independent creative agency of senior thinkers...,"Brand Strategy, Creative & Production, Marketi...",NaN
14,EV Location Partner Campaign,Allego,GH05T Ltd.,We are a social media agency operating in the ...,We are a social media agency operating in the ...,"Brand Strategy, Creative & Production, Marketi...",NaN
17,Brand Refresh,Auspaynet,Cocogun,NaN,We are a full-service independent creative age...,"Brand Strategy, Creative & Production",NaN
19,Brand Refresh,Auspaynet,Principals,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
244,Brand positioning,TotalEnergies,Flume,"Welcome to Flume, an independent research agency.","Welcome to Flume, an independent research agency.",Research & Insight,NaN
245,Brand positioning,TotalEnergies,The Nursery Research & Planning Ltd,"Specialists in brand, communications and exper...","Specialists in brand, communications and exper...",Research & Insight,NaN
246,Social Media Veezu,Veezu,GH05T Ltd.,We are a social media agency operating in the ...,We are a social media agency operating in the ...,"Brand Strategy, Creative & Production, Marketi...",NaN
249,SME engagement,Wave,NMD+,NaN,NaN,NaN,NaN


# Preprocessing and feature engineering 

In [81]:
# treating for the agency column headers
# breif taxonomy is already treated for 
def process_tags(tags):
    if pd.isnull(tags): # If the value is NaN, return it as is
        return tags
    tags = tags.split(',')
    processed_tags = []
    for tag in tags:
        tag = tag.strip() # Remove leading/trailing whitespace
        tag = tag.replace(' ', '_') # Replace spaces with underscores
        tag = '(' + tag + ')' # Put parentheses around the tag
        processed_tags.append(tag)
    return ', '.join(processed_tags)

# Apply the process_tags function to the relevant columns
columns_to_process = ['Agency Skills', 'Agency Industries', 
                      'Agency services', 'Other Services']
for col in columns_to_process:
    MERGED[col] = MERGED[col].apply(process_tags)

MERGED.head()

,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Vision,...,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Sub Brief Taxonomy,Main Brief Taxonomy
0,Awards applications x5,AXA,Boost Awards,"(Copywriting), (PR)",(Professional_Services),We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,(Public_Relations),(Awards_consultancy),NaN,...,0,0,0,0,0,0,1,0,"(PR), (Corporate_Communication)",(Media_Public Relations & Events)
1,Awards applications x5,AXA,Reed Words,"(Brand_Strategy), (Copywriting), (Corporate_Co...","(Administrative_and_Support_Services), (Alcoho...",Verbal branding and copy writing,We make brands and business stronger through l...,"(Marketing_Planning), (Social_&_Content)","(Copywriting), (Naming), (Tone_of_Voice), (Bra...",Making Brands and Businesses Stronger Through ...,...,0,0,0,0,0,0,1,0,"(PR), (Corporate_Communication)",(Media_Public Relations & Events)
2,Countrywide broker video,AXA,Viewpoint Studios,"(Brand_Design), (Customer_Experience_Design), ...","(Alcoholic_Beverages), (Automotive), (Beauty),...",Advertising and commercial photography and film,Inspired photography and film,"(Creative_&_Production), (Social_&_Content)",(Advertising_and_commercial_photography_and_film),NaN,...,0,0,0,0,0,1,0,0,(Video_Production),"(Production Services), (Advertising_Brand & Cr..."
3,Countrywide broker video,AXA,Shoreditch Design Studio,"(Brand_Design), (UX_&_UI_Design), (Mobile_App_...",(B2B),The creative agency for your next big thing,The creative agency for your next big thing,"(Strategic_Design), (UX/UI_Design)","(Product_Design), (Print_Design), (Animation),...",We create high quality digital design for busi...,...,0,0,0,0,0,1,0,0,(Video_Production),"(Production Services), (Advertising_Brand & Cr..."
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,"(Business_Strategy), (CRM_Strategy), (Customer...","(Alcoholic_Beverages), (Beauty), (Education), ...",We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"(Brand_Strategy), (Proposition_Design), (CRM_&...",NaN,Specialists in brand strategy and proposition ...,...,0,0,0,0,0,0,1,0,(Proposition_Design),(Experience & Design)


In [82]:
MERGED.head()

,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Vision,...,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough,Sub Brief Taxonomy,Main Brief Taxonomy
0,Awards applications x5,AXA,Boost Awards,"(Copywriting), (PR)",(Professional_Services),We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,(Public_Relations),(Awards_consultancy),NaN,...,0,0,0,0,0,0,1,0,"(PR), (Corporate_Communication)",(Media_Public Relations & Events)
1,Awards applications x5,AXA,Reed Words,"(Brand_Strategy), (Copywriting), (Corporate_Co...","(Administrative_and_Support_Services), (Alcoho...",Verbal branding and copy writing,We make brands and business stronger through l...,"(Marketing_Planning), (Social_&_Content)","(Copywriting), (Naming), (Tone_of_Voice), (Bra...",Making Brands and Businesses Stronger Through ...,...,0,0,0,0,0,0,1,0,"(PR), (Corporate_Communication)",(Media_Public Relations & Events)
2,Countrywide broker video,AXA,Viewpoint Studios,"(Brand_Design), (Customer_Experience_Design), ...","(Alcoholic_Beverages), (Automotive), (Beauty),...",Advertising and commercial photography and film,Inspired photography and film,"(Creative_&_Production), (Social_&_Content)",(Advertising_and_commercial_photography_and_film),NaN,...,0,0,0,0,0,1,0,0,(Video_Production),"(Production Services), (Advertising_Brand & Cr..."
3,Countrywide broker video,AXA,Shoreditch Design Studio,"(Brand_Design), (UX_&_UI_Design), (Mobile_App_...",(B2B),The creative agency for your next big thing,The creative agency for your next big thing,"(Strategic_Design), (UX/UI_Design)","(Product_Design), (Print_Design), (Animation),...",We create high quality digital design for busi...,...,0,0,0,0,0,1,0,0,(Video_Production),"(Production Services), (Advertising_Brand & Cr..."
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,"(Business_Strategy), (CRM_Strategy), (Customer...","(Alcoholic_Beverages), (Beauty), (Education), ...",We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"(Brand_Strategy), (Proposition_Design), (CRM_&...",NaN,Specialists in brand strategy and proposition ...,...,0,0,0,0,0,0,1,0,(Proposition_Design),(Experience & Design)


In [83]:
# # SAVING THE MERGED FINAL MERGED FILE
# MERGED.to_csv("MERGED.csv", index= False)

## for dataset description:
'Brief Name' : client brief name

'Client': client name

'Shortlisted Agency': agency Studiospace shorlisted for the client

'Agency SKills': skills of the shorlisted agency (categorical variable)

'Agency Industries': industry specialisations of the shorlisted agency (categorical variable)

'Service Description': service description of the agency in words

'Agency Description': short description of the agency in words

'Agency services': service description of the agency in words

'Other Services': any other service description of the agency in words

'Agency Locations': locations agency functions in (categorical variable)

'Brief Region': region of the brief, that is, region the client is from (categorical variable)

'Brief Category': brief tags (categorical variable)

'Brief Tags': briefs tagged by studiospace (categorical variable)

'Status': status of the project

'Winning Agency': 

'Number of Agencies Shortlisted': number of agencies shortlisted for that particular client brief

'Project extension with same agency': reason 1 for matching

'Client selected agency themselves from marketplace': reason 2 for matching

'Right geography / time zone': reason 3 for matching

'Strong skill set match': reason 4 for matching

'Deep / relevant sector expertise': reason 5 for matching

'Good cultural fit': reason 6 for matching

'Right price level': reason 7 for matching

'Strong track record': reason 8 for matching

'Match Score': score calculated using weights assigned to the reasons for shortlisting an agency for a particular client.


In [64]:
client_agency_rfm.head()

,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Agency Locations,...,Client_WA_Interaction,Project extension with same agency,Client selected agency themselves from marketplace,Right geography / time zone,Strong skill set match,Deep / relevant sector expertise,Good cultural fit,Right price level,Strong track record,Match Score
0,Awards applications x5,AXA,Boost Awards,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,United Kingdom,...,1.0,0,0.0,1,1,0,0,0,1,0.418605
1,Awards applications x5,AXA,Reed Words,"Brand Strategy, Copywriting, Corporate Communi...","Administrative and Support Services, Alcoholic...",Verbal branding and copy writing,We make brands and business stronger through l...,"Marketing Planning, Social & Content","Copywriting, Naming, Tone of Voice, Brand Voic...","United Kingdom, United States",...,1.0,0,0.0,1,1,0,0,0,1,0.418605
2,Countrywide broker video,AXA,Viewpoint Studios,"Brand Design, Customer Experience Design, Grap...","Alcoholic Beverages, Automotive, Beauty, Const...",Advertising and commercial photography and film,Inspired photography and film,"Creative & Production, Social & Content",Advertising and commercial photography and film,United Kingdom,...,1.0,0,0.0,0,1,0,0,1,0,0.372093
3,Countrywide broker video,AXA,Shoreditch Design Studio,"Brand Design, UX & UI Design, Mobile App Devel...",B2B,The creative agency for your next big thing,The creative agency for your next big thing,"Strategic Design, UX/UI Design","Product Design, Print Design, Animation, Illus...","Australia, United Kingdom, United States",...,1.0,0,0.0,0,1,0,0,1,1,0.465116
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,"Business Strategy, CRM Strategy, Customer Stra...","Alcoholic Beverages, Beauty, Education, Entert...",We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"Brand Strategy, Proposition Design, CRM & Loyalty",NaN,United Kingdom,...,1.0,0,1.0,1,1,0,0,0,0,0.348837


In [58]:
# Checking for duplicates in the DataFrame
duplicates = client_agency_rfm.duplicated()
print(f"Number of duplicate rows = {duplicates.sum()}")

# Checking for columns with NaN values
cols_with_nan = client_agency_rfm.isnull().sum()
cols_with_nan = cols_with_nan[cols_with_nan > 0]

# Printing the number of NaNs for each column
for col_name, nan_count in cols_with_nan.items():
    print(f"The column '{col_name}' has {nan_count} NaN values.")

# Checking data for NaNs
nan_data = client_agency_rfm[client_agency_rfm.isna().any(axis=1)]
nan_data

Number of duplicate rows = 0
The column 'Agency Skills' has 69 NaN values.
The column 'Agency Industries' has 69 NaN values.
The column 'Service Description' has 60 NaN values.
The column 'Agency Description' has 59 NaN values.
The column 'Agency services' has 59 NaN values.
The column 'Other Services' has 120 NaN values.
The column 'Vision' has 141 NaN values.
The column 'Industry Experience' has 144 NaN values.
The column 'Agency Locations' has 59 NaN values.


,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Vision,...,Strong track record,Match Score,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough
0,Awards applications x5,AXA,Boost Awards,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,NaN,...,1,0.418605,0,0,0,0,0,0,1,0
2,Countrywide broker video,AXA,Viewpoint Studios,"Brand Design, Customer Experience Design, Grap...","Alcoholic Beverages, Automotive, Beauty, Const...",Advertising and commercial photography and film,Inspired photography and film,"Creative & Production, Social & Content",Advertising and commercial photography and film,NaN,...,0,0.372093,0,0,0,0,0,1,0,0
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,"Business Strategy, CRM Strategy, Customer Stra...","Alcoholic Beverages, Beauty, Education, Entert...",We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"Brand Strategy, Proposition Design, CRM & Loyalty",NaN,Specialists in brand strategy and proposition ...,...,0,0.348837,0,0,0,0,0,0,1,0
5,Prop Dev (EV and CC Insurance),AXA,Daughter Studio,"Business Strategy, Customer Strategy, Qualitat...","Alcoholic Beverages, Automotive, Beauty, Educa...",Independent creative agency of senior thinkers...,Independent creative agency of senior thinkers...,"Brand Strategy, Creative & Production, Marketi...",NaN,We give brands the impact they deserve by sepa...,...,1,0.418605,0,0,0,0,0,0,1,0
6,Prop Dev (EV and CC Insurance),AXA,CreateFuture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0.418605,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Social Media Veezu,Veezu,GH05T Ltd.,"Data Analytics, Advertising, Brand Strategy, S...","Alcoholic Beverages, Beauty, Entertainment, Fa...",We are a social media agency operating in the ...,We are a social media agency operating in the ...,"Brand Strategy, Creative & Production, Marketi...",NaN,NaN,...,1,0.604651,0,0,0,0,1,0,0,0
247,Social Media Veezu,Veezu,Nonsensical,"Advertising, Social Media, Creative Production...","Beauty, Education, Entertainment, Fashion, Fin...",We do TikTok so you don't have to!,We do TikTok so you don't have to!,"Creative & Production, Social & Content, Digit...",TikTok,NaN,...,0,0.139535,0,0,0,0,1,0,0,0
248,Social Media Veezu,Veezu,Spin Brands,"Digital Strategy, Growth & Innovation Strategy...","Administrative and Support Services, Alcoholic...",Voted UK's #1 Social Agency,Voted UK's #1 Social Agency,"Creative & Production, Marketing Planning, Soc...","Social Media Management, Advertising + Content",NaN,...,1,0.325581,0,0,0,0,1,0,0,0
249,SME engagement,Wave,NMD+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.023256,0,0,0,0,1,0,0,0


### filling NaNs for agency skills and agency industries
### given by studiospace after rectification ask

In [51]:
# #downloading the nan data
# nan_data.to_csv("nan_data.csv")

In [65]:
client_agency_rfm

,Brief Name,Client,Status,Shortlisted Agency,Winning Agency,Brief Region,Number of Agencies Shortlisted Brief,Client_WA_Interaction,Project extension with same agency,Client selected agency themselves from marketplace,...,Right price level,Strong track record,Match Score,Agency SKills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Agency Locations
0,awards applications x5,axa,started,boost awards,boost awards,united kingdom,2,1.0,0,0.0,...,0,1,0.418605,"copywriting , pr",professional services,world ’ first largest award entry consultancy,world ’ first largest award entry consultancy,public relation,award consultancy,united kingdom
1,awards applications x5,axa,started,reed words,boost awards,united kingdom,2,1.0,0,0.0,...,0,1,0.418605,"brand strategy , copywriting , corporate commu...","administrative support services , alcoholic be...",verbal branding copy writing,make brand business stronger language,marketing planning social content,copywriting naming tone voice brand voice verb...,"united kingdom , united states"
2,countrywide broker video,axa,archived,viewpoint studios,shoreditch design studio,united kingdom,2,1.0,0,0.0,...,1,0,0.372093,"brand design , customer experience design , gr...","alcoholic beverages , automotive , beauty , co...",advertising commercial photography film,inspired photography film,creative production social content,advertising commercial photography film,united kingdom
3,countrywide broker video,axa,archived,shoreditch design studio,shoreditch design studio,united kingdom,2,1.0,0,0.0,...,1,1,0.465116,"brand design , ux ui design , mobile app devel...",b2b,creative agency next big thing,creative agency next big thing,strategic design uxui design,product design print design animation illustra...,"australia , united kingdom , united states"
4,prop dev ev cc insurance,axa,started,blue feathers,daughter studio,united kingdom,3,1.0,0,1.0,...,0,0,0.348837,"business strategy , crm strategy , customer st...","alcoholic beverages , beauty , education , ent...",work ambitious marketing leader team help navi...,work ambitious marketing leader team help navi...,brand strategy proposition design crm loyalty,NaN,united kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,brand positioning,totalenergies,matching,nursery research planning ltd,unknown,united kingdom,4,1.0,0,0.0,...,1,1,0.790698,"social media , video production",b2b,specialist brand communication experience rese...,specialist brand communication experience rese...,research insight,NaN,united kingdom
228,social media veezu,veezu,started,nonsensical,gh05t ltd .,united kingdom,3,2.0,0,0.0,...,1,0,0.139535,"advertising , social media , creative producti...","beauty , education , entertainment , fashion ,...",tiktok dont,tiktok dont,creative production social content digital mar...,tiktok,united kingdom
229,social media veezu,veezu,started,spin brands,gh05t ltd .,united kingdom,3,2.0,0,0.0,...,0,1,0.325581,"digital strategy , growth innovation strategy ...","administrative support services , alcoholic be...",voted uk social agency,voted uk social agency,creative production marketing planning social ...,social medium management advertising content,united kingdom
230,sme engagement,wave,archived,nmd,nmd,united kingdom,1,2.0,0,1.0,...,0,0,0.023256,"business strategy , customer strategy , digita...",financial services,creative technology consultancy,specialist agency focused helping company buil...,uxui design strategic design social content,specalists financial service industry,united kingdom


In [58]:
# ## downloading the file with the winning agency 
#....and the other columns which have been dropped below for embedding models
# client_agency_rfm.to_csv("embedding.csv")

In [66]:
# column_to_drop = ["Agency Description", "Agency services",
#                   "Other Services",
#                   "Client_WA_Interaction",
#                   "Service Description", "Status", "Winning Agency",
#                   "Number of Agencies Shortlisted Brief"]

# client_agency_rfm = client_agency_rfm.drop(column_to_drop, axis=1)


In [70]:
# #downloading the merged file 
# client_agency_rfm.to_csv("matching.csv")

In [59]:
client_agency_rfm

,Brief Name,Client,Shortlisted Agency,Agency Skills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Vision,...,Strong track record,Match Score,Lead_George Patten,Lead_Gideon Hyde,Lead_Hannah Fraser,Lead_Kate Walker,Lead_Paul Bowman,Lead_Peter Sayburn,Lead_Phil Kohler,Lead_Robin Scarborough
0,Awards applications x5,AXA,Boost Awards,"Copywriting, PR",Professional Services,We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,Public Relations,Awards consultancy,NaN,...,1,0.418605,0,0,0,0,0,0,1,0
1,Awards applications x5,AXA,Reed Words,"Brand Strategy, Copywriting, Corporate Communi...","Administrative and Support Services, Alcoholic...",Verbal branding and copy writing,We make brands and business stronger through l...,"Marketing Planning, Social & Content","Copywriting, Naming, Tone of Voice, Brand Voic...",Making Brands and Businesses Stronger Through ...,...,1,0.418605,0,0,0,0,0,0,1,0
2,Countrywide broker video,AXA,Viewpoint Studios,"Brand Design, Customer Experience Design, Grap...","Alcoholic Beverages, Automotive, Beauty, Const...",Advertising and commercial photography and film,Inspired photography and film,"Creative & Production, Social & Content",Advertising and commercial photography and film,NaN,...,0,0.372093,0,0,0,0,0,1,0,0
3,Countrywide broker video,AXA,Shoreditch Design Studio,"Brand Design, UX & UI Design, Mobile App Devel...",B2B,The creative agency for your next big thing,The creative agency for your next big thing,"Strategic Design, UX/UI Design","Product Design, Print Design, Animation, Illus...",We create high quality digital design for busi...,...,1,0.465116,0,0,0,0,0,1,0,0
4,Prop Dev (EV and CC Insurance),AXA,Blue Feathers,"Business Strategy, CRM Strategy, Customer Stra...","Alcoholic Beverages, Beauty, Education, Entert...",We work with ambitious marketing leaders and t...,We work with ambitious marketing leaders and t...,"Brand Strategy, Proposition Design, CRM & Loyalty",NaN,Specialists in brand strategy and proposition ...,...,0,0.348837,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Social Media Veezu,Veezu,GH05T Ltd.,"Data Analytics, Advertising, Brand Strategy, S...","Alcoholic Beverages, Beauty, Entertainment, Fa...",We are a social media agency operating in the ...,We are a social media agency operating in the ...,"Brand Strategy, Creative & Production, Marketi...",NaN,NaN,...,1,0.604651,0,0,0,0,1,0,0,0
247,Social Media Veezu,Veezu,Nonsensical,"Advertising, Social Media, Creative Production...","Beauty, Education, Entertainment, Fashion, Fin...",We do TikTok so you don't have to!,We do TikTok so you don't have to!,"Creative & Production, Social & Content, Digit...",TikTok,NaN,...,0,0.139535,0,0,0,0,1,0,0,0
248,Social Media Veezu,Veezu,Spin Brands,"Digital Strategy, Growth & Innovation Strategy...","Administrative and Support Services, Alcoholic...",Voted UK's #1 Social Agency,Voted UK's #1 Social Agency,"Creative & Production, Marketing Planning, Soc...","Social Media Management, Advertising + Content",NaN,...,1,0.325581,0,0,0,0,1,0,0,0
249,SME engagement,Wave,NMD+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.023256,0,0,0,0,1,0,0,0


### checking for agency headers since some have been removed (went from 280 to 196, have to check properly after with SS)

In [72]:
def check_value_in_column(dataframe, column_name, value_to_check):
    if value_to_check in dataframe[column_name].values:
        return True
    else:
        return False

In [73]:
check_value_in_column(agency, "Agency", "flume")

True

In [77]:
# # Create a new dataframe without NaN values in "Skills" and "Industries" columns
# matching = client_agency_rfm.dropna(subset=['Agency SKills', 'Agency Industries'])
# matching

,Brief Name,Client,Shortlisted Agency,Agency SKills,Agency Industries,Service Description,Agency Description,Agency services,Other Services,Agency Locations,...,Client_WA_Interaction,Project extension with same agency,Client selected agency themselves from marketplace,Right geography / time zone,Strong skill set match,Deep / relevant sector expertise,Good cultural fit,Right price level,Strong track record,Match Score
0,awards applications x5,axa,boost awards,"copywriting , pr",professional services,world ’ first largest award entry consultancy,world ’ first largest award entry consultancy,public relation,award consultancy,united kingdom,...,1.0,0,0.0,1,1,0,0,0,1,0.418605
1,awards applications x5,axa,reed words,"brand strategy , copywriting , corporate commu...","administrative support services , alcoholic be...",verbal branding copy writing,make brand business stronger language,marketing planning social content,copywriting naming tone voice brand voice verb...,"united kingdom , united states",...,1.0,0,0.0,1,1,0,0,0,1,0.418605
2,countrywide broker video,axa,viewpoint studios,"brand design , customer experience design , gr...","alcoholic beverages , automotive , beauty , co...",advertising commercial photography film,inspired photography film,creative production social content,advertising commercial photography film,united kingdom,...,1.0,0,0.0,0,1,0,0,1,0,0.372093
3,countrywide broker video,axa,shoreditch design studio,"brand design , ux ui design , mobile app devel...",b2b,creative agency next big thing,creative agency next big thing,strategic design uxui design,product design print design animation illustra...,"australia , united kingdom , united states",...,1.0,0,0.0,0,1,0,0,1,1,0.465116
4,prop dev ev cc insurance,axa,blue feathers,"business strategy , crm strategy , customer st...","alcoholic beverages , beauty , education , ent...",work ambitious marketing leader team help navi...,work ambitious marketing leader team help navi...,brand strategy proposition design crm loyalty,NaN,united kingdom,...,1.0,0,1.0,1,1,0,0,0,0,0.348837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,brand positioning,totalenergies,nursery research planning ltd,"social media , video production",b2b,specialist brand communication experience rese...,specialist brand communication experience rese...,research insight,NaN,united kingdom,...,1.0,0,0.0,1,1,1,0,1,1,0.790698
228,social media veezu,veezu,nonsensical,"advertising , social media , creative producti...","beauty , education , entertainment , fashion ,...",tiktok dont,tiktok dont,creative production social content digital mar...,tiktok,united kingdom,...,2.0,0,0.0,0,0,0,0,1,0,0.139535
229,social media veezu,veezu,spin brands,"digital strategy , growth innovation strategy ...","administrative support services , alcoholic be...",voted uk social agency,voted uk social agency,creative production marketing planning social ...,social medium management advertising content,united kingdom,...,2.0,0,0.0,0,1,0,0,0,1,0.325581
230,sme engagement,wave,nmd,"business strategy , customer strategy , digita...",financial services,creative technology consultancy,specialist agency focused helping company buil...,uxui design strategic design social content,specalists financial service industry,united kingdom,...,2.0,0,1.0,0,0,0,0,0,0,0.023256


In [79]:
# column_to_drop = ["Agency Description", "Agency services", 
#                   'Other Services',
#                   "Client_WA_Interaction",
#                  "Service Description", "Status", "Winning Agency", "Number of Agencies Shortlisted Brief"]
# matching = matching.drop(column_to_drop, axis=1)
# matching

,Brief Name,Client,Shortlisted Agency,Agency SKills,Agency Industries,Agency Locations,Brief Region,Brief Category,Brief Tags,Project extension with same agency,Client selected agency themselves from marketplace,Right geography / time zone,Strong skill set match,Deep / relevant sector expertise,Good cultural fit,Right price level,Strong track record,Match Score
0,awards applications x5,axa,boost awards,"copywriting , pr",professional services,united kingdom,united kingdom,pr communications,rol optimisation,0,0.0,1,1,0,0,0,1,0.418605
1,awards applications x5,axa,reed words,"brand strategy , copywriting , corporate commu...","administrative support services , alcoholic be...","united kingdom , united states",united kingdom,pr communications,rol optimisation,0,0.0,1,1,0,0,0,1,0.418605
2,countrywide broker video,axa,viewpoint studios,"brand design , customer experience design , gr...","alcoholic beverages , automotive , beauty , co...",united kingdom,united kingdom,video,"photography , events , data analytics",0,0.0,0,1,0,0,1,0,0.372093
3,countrywide broker video,axa,shoreditch design studio,"brand design , ux ui design , mobile app devel...",b2b,"australia , united kingdom , united states",united kingdom,video,"photography , events , data analytics",0,0.0,0,1,0,0,1,1,0.465116
4,prop dev ev cc insurance,axa,blue feathers,"business strategy , crm strategy , customer st...","alcoholic beverages , beauty , education , ent...",united kingdom,united kingdom,proposition design,"web development , advertising",0,1.0,1,1,0,0,0,0,0.348837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,brand positioning,totalenergies,nursery research planning ltd,"social media , video production",b2b,united kingdom,united kingdom,research insight,digital strategy,0,0.0,1,1,1,0,1,1,0.790698
228,social media veezu,veezu,nonsensical,"advertising , social media , creative producti...","beauty , education , entertainment , fashion ,...",united kingdom,united kingdom,social content,quantitative research,0,0.0,0,0,0,0,1,0,0.139535
229,social media veezu,veezu,spin brands,"digital strategy , growth innovation strategy ...","administrative support services , alcoholic be...",united kingdom,united kingdom,social content,quantitative research,0,0.0,0,1,0,0,0,1,0.325581
230,sme engagement,wave,nmd,"business strategy , customer strategy , digita...",financial services,united kingdom,united kingdom,proposition design,"graphic design , ecommerce marketplaces , expe...",0,1.0,0,0,0,0,0,0,0.023256


## for dataset description:
'Brief Name' : client brief name

'Client': client name

'Shortlisted Agency': agency Studiospace shorlisted for the client

'Agency SKills': skills of the shorlisted agency (categorical variable)

'Agency Industries': industry specialisations of the shorlisted agency (categorical variable)

'Agency Locations': locations agency functions in (categorical variable)

'Brief Region': region of the brief, that is, region the client is from (categorical variable)

'Brief Tags': briefs tagged by studiospace (categorical variable)

'Project extension with same agency': reason 1 for matching

'Client selected agency themselves from marketplace': reason 2 for matching

'Right geography / time zone': reason 3 for matching

'Strong skill set match': reason 4 for matching

'Deep / relevant sector expertise': reason 5 for matching

'Good cultural fit': reason 6 for matching

'Right price level': reason 7 for matching

'Strong track record': reason 8 for matching

'Match Score': score calculated using weights assigned to the reasons for shortlisting an agency for a particular client.

"matching" here means an agency being shortlisted for a client brief. 

#### downloading the merged file to further process and work on in a cleaner file named "new_algorithm_modelling"

In [85]:
# matching.to_csv("matching.csv")

In [84]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report

# # Assuming target variable is 'Match Score'
# # Considering it as a binary classification problem (Score >= 0.5 : 1 else : 0)
# matching['Match Score'] = matching['Match Score'].apply(lambda x: 1 if x >= 0.5 else 0)

# # Define preprocessor
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('tfidf', TfidfVectorizer(), ['Brief Name', 'Shortlisted Agency', 'Agency Skills', 'Agency Industries',
#                                       'Service Description', 'Agency Description', 'Agency services',
#                                       'Agency Locations', 'Brief Region', 'Brief Category', 'Brief Tags',
#                                       'Status', 'Winning Agency']),
#         ('onehot', OneHotEncoder(), ['Number of Agencies Shortlisted Brief', 'Project extension with same agency',
#                                      'Client selected agency themselves from marketplace', 'Right geography / time zone',
#                                      'Strong skill set match', 'Deep / relevant sector expertise', 'Good cultural fit',
#                                      'Right price level', 'Strong track record']),
#         ('scale', StandardScaler(), ['Client_WA_Interaction'])
#     ])

# X = matching.drop('Match Score', axis=1)
# y = matching['Match Score']

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define pipeline
# pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                            ('classifier', RandomForestClassifier())])

# # Train the model
# pipeline.fit(X_train, y_train)

# # Test the model
# y_pred = pipeline.predict(X_test)

# # Print classification report
# print(classification_report(y_test, y_pred))

In [59]:

# # Feature Importance
# try:
#     importances = pipeline.steps[1][1].feature_importances_
#     for feature, importance in enumerate(importances):
#         print(f"Feature: {feature}, Importance: {importance}")
# except AttributeError:
#     print("The used classifier does not expose \"feature_importances_\" ")

### Multilabel classification problem-RF


In [57]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# # from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# from sklearn.pipeline import FeatureUnion, Pipeline
# from sklearn.compose import ColumnTransformer

In [58]:

# # columns to be vectorized
# text_features = ['Brief Name', 'Client', 'Brief Tags', 'Agency SKills', 
#                  'Agency Industries', 'Service Description', 'Agency Description', 
#                  'Agency services', 'Agency Locations']

# # columns to be one-hot encoded
# categorical_features = ['Brief Region', 'Brief Category', 'Status']

# # target columns
# target = ['Project extension with same agency',
#           'Client selected agency themselves from marketplace',
#           'Right geography / time zone', 
#           'Strong skill set match', 
#           'Deep / relevant sector expertise',
#           'Good cultural fit', 
#           'Right price level', 
#           'Strong track record']


In [59]:
# # separate features and target
# X = matching[text_features + categorical_features]
# y = matching[target]

# # split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [60]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class TextProcessor(BaseEstimator, TransformerMixin):
#     def __init__(self, column):
#         self.column = column

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return X[self.column]

# # create the preprocessing pipelines for both numeric and categorical data
# text_transformers = [(f'Tfidf_{col}', Pipeline(steps=[
#                                         ('selector', TextProcessor(col)),
#                                         ('tfidf', TfidfVectorizer())
#                                         ]), [col]) for col in text_features]

# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# preprocessor = ColumnTransformer(
#     transformers=text_transformers + [
#         ('cat', categorical_transformer, categorical_features)])

# # append classifier to preprocessing pipeline
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestClassifier())])

# # fit the model
# clf.fit(X_train, y_train)

# # make predictions
# y_pred = clf.predict(X_test)

# # print classification report
# print(classification_report(y_test, y_pred))

In [61]:
# from sklearn.base import BaseEstimator, TransformerMixin
# import plotly as plt

# class TextProcessor(BaseEstimator, TransformerMixin):
#     def __init__(self, column):
#         self.column = column

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return X[self.column]

# # create the preprocessing pipelines for both numeric and categorical data
# text_transformers = [(f'Tfidf_{col}', Pipeline(steps=[
#                                         ('selector', TextProcessor(col)),
#                                         ('tfidf', TfidfVectorizer())
#                                         ]), [col]) for col in text_features]

# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# preprocessor = ColumnTransformer(
#     transformers=text_transformers + [
#         ('cat', categorical_transformer, categorical_features)])

# # append classifier to preprocessing pipeline
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', RandomForestClassifier())])

# # fit the model
# clf.fit(X_train, y_train)

# # get feature names
# feature_names = []
# for name, trans, cols in clf.named_steps['preprocessor'].transformers_:
#     if name.startswith('Tfidf_'):
#         feature_names.extend([f'{name}_{col}' for col in trans.named_steps['tfidf'].get_feature_names()])
#     elif name == 'cat':
#         feature_names.extend(trans.named_steps['onehot'].get_feature_names(categorical_features))

# # make predictions
# y_pred = clf.predict(X_test)

# # print classification report
# print(classification_report(y_test, y_pred))

# # feature importance
# importances = clf.named_steps['classifier'].feature_importances_
# std = np.std([tree.feature_importances_ for tree in clf.named_steps['classifier'].estimators_], axis=0)
# forest_importances = pd.Series(importances, index=feature_names)

# fig, ax = plt.subplots()
# forest_importances.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances using MDI")
# ax.set_ylabel("Mean decrease impurity")
# fig.tight_layout()

# Deep Learning Approach-Neural Network

## Deep Learning MLP Classifier


In [59]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import classification_report
# from sklearn.neural_network import MLPClassifier

# # Create a list of column names of the "reasons" columns to drop for the test set
# reason_columns = matching.columns[16:24]

# # Create training and test sets
# X_train, X_test, y_train, y_test = train_test_split(matching.drop(columns=['Match Score', 'Project extension with same agency', 'Client selected agency themselves from marketplace', 
#                         'Right geography / time zone', 'Strong skill set match', 
#                         'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 
#                         'Strong track record']), matching['Match Score'],
#                                                     test_size=0.2, random_state=42)

# # Now, drop the "reasons" columns from the test set
# X_test = X_test.drop(columns=reason_columns)

# # Define columns for tf-idf and one-hot encoding
# text_features = ['Brief Name', 'Client', 'Shortlisted Agency', 'Agency SKills', 'Agency Industries', 
#                  'Service Description', 'Agency Description', 'Agency services', 'Agency Locations', 
#                  'Brief Region', 'Brief Category', 'Brief Tags', 'Status', 'Winning Agency']

# categorical_features = ['Number of Agencies Shortlisted Brief', 'Client_WA_Interaction'] + reason_columns.tolist()

# # Define column transformer
# preprocessor = make_column_transformer(
#     (TfidfVectorizer(), text_features),
#     (OneHotEncoder(), categorical_features)
# )

# # Define neural network classifier
# nn = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=200)

# # Make pipeline
# clf = make_pipeline(preprocessor, nn)

# # fit the model
# clf.fit(X_train, y_train)

# # make predictions
# y_pred = clf.predict(X_test)

# # print classification report
# print(classification_report(y_test, y_pred))


KeyError: "['Project extension with same agency', 'Client selected agency themselves from marketplace', 'Right geography / time zone', 'Strong skill set match', 'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 'Strong track record'] not found in axis"

In [ ]:
# matching.head()

# basic neural network

In [138]:
# features_to_drop = ['Project extension with same agency', 'Client selected agency themselves from marketplace', 
#                     'Right geography / time zone', 'Strong skill set match', 
#                     'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 
#                     'Strong track record', 'Match Score']
# df_reduced = matching.drop(columns=features_to_drop)

In [139]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df_reduced.drop(columns='Shortlisted Agency'), df_reduced['Shortlisted Agency'], test_size=0.2, random_state=42)

In [141]:
# X_train = pd.concat([X_train, matching[matching.index.isin(X_train.index)][features_to_drop]], axis=1)


In [146]:
# # One-hot encoding
# X_train_filled_encoded = pd.get_dummies(X_train_filled)

# # Convert DataFrame to float
# X_train_float = X_train_filled_encoded.astype(float)

# # Define the model
# model = Sequential()
# model.add(Dense(10, input_dim=X_train_float.shape[1], activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Fit the model
# model.fit(X_train_float, y_train, epochs=10, batch_size=32, verbose=2)


Epoch 1/10


2023-06-30 17:59:44.728371: W tensorflow/core/framework/op_kernel.cc:1722] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError:  Cast string to float is not supported
	 [[node binary_crossentropy/Cast
 (defined at /opt/anaconda/envs/Python3/lib/python3.9/site-packages/keras/losses.py:1797)
]] [Op:__inference_train_function_1471]

Errors may have originated from an input operation.
Input Source operations connected to node binary_crossentropy/Cast:
In[0] IteratorGetNext (defined at /opt/anaconda/envs/Python3/lib/python3.9/site-packages/keras/engine/training.py:866)

Operation defined at: (most recent call last)
>>>   File "/opt/anaconda/envs/Python3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/opt/anaconda/envs/Python3/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/opt/anaconda/envs/Python3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/opt/anaconda/envs/Python3/lib/python3.9/site-packa

### to check with test data

In [ ]:
# # Choose the first brief from the test set as an example
# test_brief = X_test.iloc[0].to_frame().T

# # get probabilities for each class
# probabilities = model.predict(test_brief)

# # create a DataFrame for easier manipulation
# prob_df = pd.DataFrame(probabilities, columns=model.classes_)

# # rank agencies based on probabilities
# recommended_agencies = prob_df.sort_values(by='1', ascending=False)  # replace '1' with the name of the positive class


### to check with adding a new brief (completely new data)

In [ ]:
# # this is an example, you need to replace it with your actual new brief data
# new_brief = pd.DataFrame({
#     'Brief Name': ['New Brief'],
#     'Client': ['New Client'],
#     # ...
# })

# # make sure to transform your new brief into the same format as your training data
# # if you have preprocessing steps such as text cleaning, you should apply them to your new data as well

# # get probabilities for each class
# probabilities = model.predict(new_brief)

# # create a DataFrame for easier manipulation
# prob_df = pd.DataFrame(probabilities, columns=model.classes_)

# # rank agencies based on probabilities
# recommended_agencies = prob_df.sort_values(by='1', ascending=False)  # replace '1' with the name of the positive class


In [151]:
# matching.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 0 to 173
Columns: 311 entries, Brief Name to 99
dtypes: float64(302), object(9)
memory usage: 538.7+ KB


In [60]:
## treating for categorical

In [121]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

# Choose either OneHotEncoder or LabelEncoder based on your needs
# Here we use OneHotEncoder

categorical_features = ["Brief Tags", "Agency SKills",
                        "Agency Industries", "Brief Region", 
                        "Agency Locations"]

# Generate dummy variables for comma-separated categories
for feature in categorical_features:
    # Splitting comma-separated values and strip the leading/trailing white spaces
    matching[feature] = matching[feature].str.split(',').apply(lambda x: [i.strip() for i in x])

    # 'Explode' the lists into separate rows
    exploded_feature = matching[feature].explode()

    # Get dummy variables
    dummy_features = pd.get_dummies(exploded_feature, prefix=feature).groupby(level=0).sum()

    # Concatenate dummy variables with main dataframe
    matching = pd.concat([matching, dummy_features], axis=1)

# Remove the original categorical columns from dataframe
matching.drop(categorical_features, axis=1, inplace=True)


In [122]:
matching.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196 entries, 0 to 231
Columns: 204 entries, Brief Name to Agency Locations_united states
dtypes: float64(2), int64(7), object(4), uint8(191)
memory usage: 58.0+ KB


In [62]:
# import numpy as np
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer

# text_features = ["Agency Description", "Service Description"]

# # Convert NaN values to empty strings
# # matching[text_features] = matching[text_features].fillna("")
# matching[text_features] = matching[text_features].applymap(lambda x: "" if pd.isna(x) else str(x))

# for feature in text_features:
#     tfidf = TfidfVectorizer(max_features=100)  # Limiting to 100 features. You can change this based on your needs
#     matching_tfidf = pd.DataFrame(tfidf.fit_transform(matching[feature]).toarray())
#     matching = pd.concat([matching, matching_tfidf.add_suffix('_' + feature)], axis=1)
# # # Remove the original text columns from the dataframe
# matching.drop(text_features, axis=1, inplace=True)


In [67]:
# drop_cols = ['Project extension with same agency', 'Client selected agency themselves from marketplace', 'Right geography / time zone', 'Strong skill set match', 'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 'Strong track record', 'Match Score']

# matching = matching.drop(columns=drop_cols)

In [68]:
# # drop_cols = ['Project extension with same agency', 'Client selected agency themselves from marketplace', 'Right geography / time zone', 'Strong skill set match', 'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 'Strong track record', 'Match Score']

# # matching = matching.drop(columns=drop_cols)

# from sklearn.model_selection import train_test_split

# X = matching.drop('Shortlisted Agency', axis=1)
# y = matching['Shortlisted Agency']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [68]:
# drop_cols = ['Project extension with same agency', 'Client selected agency themselves from marketplace', 'Right geography / time zone', 'Strong skill set match', 'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 'Strong track record', 'Match Score']

# matching = matching.drop(columns=drop_cols)

# from sklearn.model_selection import train_test_split

# X = matching.drop('Shortlisted Agency', axis=1)
# y = matching['Shortlisted Agency']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# # Determine the intersection of the training data indices with the indices of the rows that contain the reasons
# intersect_indices = matching.index.intersection(X_train.index)

# # Get the rows from the original dataset that are in the intersection
# intersect_rows = matching.loc[intersect_indices]

# # Add the reasons back to the training data based on the intersection
# X_train = pd.concat([X_train, intersect_rows[drop_cols]], axis=1)

# # Fill any resulting NaNs with 0
# X_train.fillna(0, inplace=True)

KeyError: "None of [Index(['Project extension with same agency',\n       'Client selected agency themselves from marketplace',\n       'Right geography / time zone', 'Strong skill set match',\n       'Deep / relevant sector expertise', 'Good cultural fit',\n       'Right price level', 'Strong track record', 'Match Score'],\n      dtype='object')] are in the [columns]"

# random forest

In [125]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# from sklearn.model_selection import train_test_split

# # Define input features and target variable
# features = [col for col in matching.columns if col not in ['Shortlisted Agency', 'Brief Name', 'Client']]
# X = matching[features]
# y = matching['Shortlisted Agency']

# # Splitting the dataset into the Training set and Test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# # Creating the Random Forest Classifier
# classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)

# # Fitting the model
# classifier.fit(X_train, y_train)

# # Predicting the Test set results
# y_pred = classifier.predict(X_test)

# # Checking the performance of the model
# print(classification_report(y_test, y_pred))


ValueError: could not convert string to float: 'creative production'

In [127]:
# Check each column in the DataFrame and print out any that contain non-numeric data
non_numeric_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
print("Non-numeric columns: ", non_numeric_cols)

# Define input features and target variable excluding non_numeric_cols
features = [col for col in matching.columns if col not in ['Shortlisted Agency', 'Brief Name', 'Client'] + non_numeric_cols]
X = matching[features]
y = matching['Shortlisted Agency']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# Creating the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)

# Fitting the model
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Checking the performance of the model
print(classification_report(y_test, y_pred))


Non-numeric columns:  []


ValueError: could not convert string to float: 'creative production'

In [128]:
# Define input features and target variable excluding 'Brief Name', 'Client'
features = [col for col in matching.columns if col not in ['Shortlisted Agency', 'Brief Name', 'Client']]

# Ensuring all data used is numerical
matching[features] = matching[features].apply(pd.to_numeric, errors='coerce')

X = matching[features]
y = matching['Shortlisted Agency']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

# Creating the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)

# Fitting the model
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Checking the performance of the model
print(classification_report(y_test, y_pred))


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [129]:
# Remove classes with single occurrence
counts = matching['Shortlisted Agency'].value_counts()
matching = matching[~matching['Shortlisted Agency'].isin(counts[counts < 2].index)]

# Define input features and target variable excluding 'Brief Name', 'Client'
features = [col for col in matching.columns if col not in ['Shortlisted Agency', 'Brief Name', 'Client']]

# Ensuring all data used is numerical
matching[features] = matching[features].apply(pd.to_numeric, errors='coerce')

X = matching[features]
y = matching['Shortlisted Agency']

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

# Creating the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)

# Fitting the model
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Checking the performance of the model
print(classification_report(y_test, y_pred))


ValueError: The test_size = 32 should be greater or equal to the number of classes = 44

In [130]:
# Define input features and target variable excluding 'Brief Name', 'Client'
features = [col for col in matching.columns if col not in ['Shortlisted Agency', 'Brief Name', 'Client']]

# Ensuring all data used is numerical
matching[features] = matching[features].apply(pd.to_numeric, errors='coerce')

X = matching[features]
y = matching['Shortlisted Agency']

# Splitting the dataset into the Training set and Test set without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Creating the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)

# Fitting the model
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Checking the performance of the model
print(classification_report(y_test, y_pred))


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [131]:
# Define input features and target variable excluding 'Brief Name', 'Client'
features = [col for col in matching.columns if col not in ['Shortlisted Agency', 'Brief Name', 'Client']]

# Ensuring all data used is numerical
matching[features] = matching[features].apply(pd.to_numeric, errors='coerce')

# Handle NaN values - Here I am filling NaNs with 0. You might want to replace it with something more suitable like mean or median
matching[features] = matching[features].fillna(0)

X = matching[features]
y = matching['Shortlisted Agency']

# Splitting the dataset into the Training set and Test set without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Creating the Random Forest Classifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)

# Fitting the model
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Checking the performance of the model
print(classification_report(y_test, y_pred))


                           precision    recall  f1-score   support

                    ampel       1.00      1.00      1.00         1
                     bang       1.00      1.00      1.00         1
              buddy media       1.00      1.00      1.00         2
          daughter studio       1.00      1.00      1.00         1
                 frontier       1.00      1.00      1.00         1
          good innovation       1.00      1.00      1.00         2
            innovate live       1.00      1.00      1.00         2
                  loomery       1.00      1.00      1.00         1
               nimbletank       1.00      1.00      1.00         1
                      nmd       1.00      1.00      1.00         1
              nonsensical       1.00      1.00      1.00         1
                     ohmy       1.00      1.00      1.00         1
orange panther collective       1.00      1.00      1.00         1
            path ventures       1.00      1.00      1.00     

In [ ]:
# ranodom code trials...

In [123]:
from sklearn.model_selection import train_test_split

# Define predictors and target variable
X = matching.drop('Shortlisted Agency', axis=1)
y = matching['Shortlisted Agency']


In [65]:

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Drop the columns from the test set
drop_cols = ['Project extension with same agency', 'Client selected agency themselves from marketplace', 
             'Right geography / time zone', 'Strong skill set match', 
             'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 
             'Strong track record', 'Match Score']
X_test = X_test.drop(columns=drop_cols)

# Fill any resulting NaNs in train set with 0
X_train.fillna(0, inplace=True)


Choosing a neural network for this problem can be justified by the fact that neural networks are able to learn complex patterns and relationships in the data. If the factors determining which agency gets shortlisted for a brief are complex and non-linear, a neural network might be able to learn these better than a simpler model.

In [68]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

# Handling categorical data and NaNs
imputer = SimpleImputer(strategy='mean')  # You might need to adjust this depending on your data

X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Convert the 'Shortlisted Agency' labels into one-hot vectors
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse=False)

y_train_encoded = label_encoder.fit_transform(y_train)
y_train_one_hot = one_hot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))

y_test_encoded = label_encoder.transform(y_test)
y_test_one_hot = one_hot_encoder.transform(y_test_encoded.reshape(-1, 1))

# Define the model architecture
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train_imputed.shape[1],)))
model.add(Dense(y_train_one_hot.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_imputed, y_train_one_hot, epochs=20, batch_size=32, verbose=1)


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'prop dev ev cc insurance'

## ^ above is the main error im facing

In [67]:
matching

,Brief Name,Client,Shortlisted Agency,Brief Category,Project extension with same agency,Client selected agency themselves from marketplace,Right geography / time zone,Strong skill set match,Deep / relevant sector expertise,Good cultural fit,...,Agency Industries_telecommunications,Agency Industries_transportation,Agency Industries_travel tourism,Agency Industries_utilities,Brief Region_australia,Brief Region_united kingdom,Brief Region_united states,Agency Locations_australia,Agency Locations_united kingdom,Agency Locations_united states
0,awards applications x5,axa,boost awards,pr communications,0,0.0,1,1,0,0,...,0,0,0,0,0,1,0,0,1,0
1,awards applications x5,axa,reed words,pr communications,0,0.0,1,1,0,0,...,1,0,1,1,0,1,0,0,1,1
2,countrywide broker video,axa,viewpoint studios,video,0,0.0,0,1,0,0,...,1,1,1,0,0,1,0,0,1,0
3,countrywide broker video,axa,shoreditch design studio,video,0,0.0,0,1,0,0,...,0,0,0,0,0,1,0,1,1,1
4,prop dev ev cc insurance,axa,blue feathers,proposition design,0,1.0,1,1,0,0,...,0,0,1,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,brand positioning,totalenergies,nursery research planning ltd,research insight,0,0.0,1,1,1,0,...,0,0,0,0,0,1,0,0,1,0
228,social media veezu,veezu,nonsensical,social content,0,0.0,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
229,social media veezu,veezu,spin brands,social content,0,0.0,0,1,0,0,...,1,1,1,1,0,1,0,0,1,0
230,sme engagement,wave,nmd,proposition design,0,1.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [70]:
# Predict probabilities for each agency
probs = model.predict(X_new)  # X_new is your new brief

# Create a dataframe of agencies and their predicted probabilities
recommendations = pd.DataFrame({'Agency': matching['Shortlisted Agency'], 'Probability': probs})

# Sort by probability
recommendations.sort_values('Probability', ascending=False, inplace=True)

# Display the top 5 recommended agencies
recommendations.head(5)


NameError: name 'model' is not defined

In [71]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from keras.models import Sequential
# from keras.layers import Dense

# # Define predictors and target variable
# X = matching.drop('Shortlisted Agency', axis=1)
# y = matching['Shortlisted Agency']

# # Split into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Drop the columns from the test set
# drop_cols = ['Project extension with same agency', 'Client selected agency themselves from marketplace', 
#              'Right geography / time zone', 'Strong skill set match', 
#              'Deep / relevant sector expertise', 'Good cultural fit', 'Right price level', 
#              'Strong track record', 'Match Score']
# X_train = X_train.drop(columns=drop_cols)
# X_test = X_test.drop(columns=drop_cols)

# Check for any categorical features and handle them appropriately here

# Fill any resulting NaNs in train and test set with 0
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

# You might want to normalize/standardize your data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the constructor
model = Sequential()

# Add an input layer and a hidden layer
model.add(Dense(12, activation='relu', input_shape=(X_train.shape[1],)))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)


ValueError: could not convert string to float: 'prop dev ev cc insurance'

In [72]:
# Check if there are any non-numeric data in the DataFrame
non_numeric = X_train.select_dtypes(exclude=[np.number])

# Print out these non-numeric columns
print(non_numeric.columns)

Index(['Brief Name', 'Client', 'Brief Category'], dtype='object')


In [73]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Parameters
num_briefs = len(set(train_data[:, 0]))  # Number of unique briefs
num_agencies = len(set(train_data[:, 1]))  # Number of unique agencies
embedding_dim = 50  # Dimension of the embedding space

# Define the model
brief_input = Input(shape=(1,), dtype='int32')
brief_embedding = Embedding(input_dim=num_briefs, output_dim=embedding_dim, name='brief_embedding')(brief_input)
brief_vec = Flatten()(brief_embedding)

agency_input = Input(shape=(1,), dtype='int32')
agency_embedding = Embedding(input_dim=num_agencies, output_dim=embedding_dim, name='agency_embedding')(agency_input)
agency_vec = Flatten()(agency_embedding)

concatenated = Concatenate()([brief_vec, agency_vec])
dropout = Dropout(0.2)(concatenated)
hidden = Dense(64, activation='relu')(dropout)
output = Dense(1, activation='sigmoid')(hidden)

model = Model(inputs=[brief_input, agency_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([train_data[:, 0], train_data[:, 1]], train_labels, 
          validation_data=([test_data[:, 0], test_data[:, 1]], test_labels), 
          epochs=10, batch_size=64)

# Make predictions
predictions = model.predict([test_data[:, 0], test_data[:, 1]])

NameError: name 'train_data' is not defined